In [1]:
import spacy
from spacy import displacy
import random
import json
import logging
import pickle

### Loading Annotated Medical Letter document from Dataturks

### Script to convert dataturks json file to spacy required format
**Present format**:
"annotation": [{
        "label": ["Form"],
        "points": [{
            "start": 8151,
            "end": 8156,
            "text": "tablet"
        }]
    }, {
        "label": ["Drug"],
        "points": [{
            "start": 8137,
            "end": 8148,
            "text": "darolutamide"
        }]
    }, {
        "label": ["Adverse Event"],
        "points": [{
            "start": 7958,
            "end": 7965,
            "text": "asthenia"
        }]
    }

**Required format**:
{'entities': [(8151, 8157, 'Form'),
    (8137, 8149, 'Drug'),
    (7958, 7966, 'Adverse Event'),
    (7950, 7957, 'Adverse Event'),
    (7771, 7778, 'Form'),
    (7701, 7707, 'Form'),
    (7572, 7584, 'Drug'),
    (7511, 7520, 'Duration'),
    (7028, 7034, 'Condition'),
    (6936, 6942, 'Strength'),
    (6909, 6921, 'Drug'),

In [55]:
# def convert_dataturks_to_spacy(data):
#     try:
#         training_data = []
#         lines=[]
#         # with open(filename, 'r') as f:
#         #     lines = f.readlines()

#         # for line in lines:
#         # data = json.loads(dataturks_data)
#         text = data['content']
#         entities = []
#         for annotation in data['annotation']:
#             #only a single point in text annotation.
#             point = annotation['points'][0]
#             labels = annotation['label']
#             # handle both list of labels or a single label.
#             if not isinstance(labels, list):
#                 labels = [labels]

#             for label in labels:
#                 #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
#                 entities.append((point['start'], point['end'] + 1 ,label))


#         training_data.append((text, {"entities" : entities}))

#         return training_data
#     except Exception as e:
#         logging.exception("Unable to process " + "\n" + "error = " + str(e))
#         return None

In [2]:
def convertSingleItem(dataturks_labeled_item):
    try:
        data = json.loads(dataturks_labeled_item)
        text = data['content']
        entities = []
        for annotation in data['annotation']:
            #only a single point in text annotation.
            point = annotation['points'][0]
            labels = annotation['label']
            # handle both list of labels or a single label.
            if not isinstance(labels, list):
                labels = [labels]

            for label in labels:
                #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                entities.append((point['start'], point['end'] + 1 ,label));

        return (text, {"entities" : entities});

    except Exception as e:
        logging.exception("Unable to process item " + dataturks_labeled_item + "\n" + "error = " + str(e))
        return None

In [3]:
training_output_FilePath = './data/Train_data'
dataturks_JSON_FilePath = './data/DrugNER.json'

In [4]:
def convert_dataturks_to_spacy(dataturks_JSON_FilePath, training_output_FilePath):
    with open(training_output_FilePath, "a+") as f:
        logging.info("File " + training_output_FilePath + " exists....")

    logging.info("Converting " + dataturks_JSON_FilePath + " ...");

    lines = []
    with open(dataturks_JSON_FilePath, 'r') as f:
        lines = f.readlines()

    if (not lines or len(lines) == 0):
        logging.exception(
            "Please specify a valid path to dataturks JSON output file, " + dataturks_JSON_FilePath + " is empty")
        return

    count = 0;
    success = 0
    training_data = []
    for line in lines:
        result = convertSingleItem(line)
        if (result):
            training_data.append(result)
            success = success + 1

        count += 1;
        if (count % 100 == 0):
            logging.info(str(count) + " items done ...")


    with open(training_output_FilePath, 'wb') as output:
        pickle.dump(training_data, output, pickle.HIGHEST_PROTOCOL)

    logging.info(
        "Completed: " + str(success) + " items done, " + str(len(lines) - success) + " items ignored due to errors")
    
    return training_data

### Sanity check for format conversion

In [6]:
TRAIN_DATA = convert_dataturks_to_spacy(dataturks_JSON_FilePath, training_output_FilePath)
TRAIN_DATA

[('The dose should be prescribed using a combination of whole tablet strengths (62.5 mg, 125 mg, 250 mg, 375 mg).',
  {'entities': [(76, 81, 'Strength'),
    (82, 85, 'Strength'),
    (86, 89, 'Strength'),
    (90, 93, 'Strength'),
    (94, 97, 'Strength'),
    (98, 101, 'Strength'),
    (102, 105, 'Strength'),
    (106, 110, 'Strength')]}),
 ('Dogs   The recommended dosage of CLAVAMOX CHEWABLE Tablet is 6.25 mg/lb of body weight twice a day.',
  {'entities': [(61, 65, 'Dosage'),
    (66, 71, 'Dosage'),
    (87, 92, 'Frequency'),
    (93, 94, 'Frequency'),
    (95, 99, 'Frequency')]}),
 ('Administer CERENIA Tablets orally at a minimum dose of 2 mg/kg (0.9 mg/lb) body weight once daily for up to 5 consecutive days',
  {'entities': [(63, 67, 'Dosage'),
    (68, 74, 'Dosage'),
    (87, 91, 'Frequency'),
    (92, 97, 'Frequency')]}),
 ('CERENIA Injectable Solution is recommended at a dose of 1 mg/kg once daily.',
  {'entities': [(56, 57, 'Dosage'),
    (58, 63, 'Dosage'),
    (64, 68, 'Fre

In [2]:
# def train_spacy(training_pickle_file):
    
#     #read pickle file to load training data
#     with open(training_pickle_file, 'rb') as input:
#         TRAIN_DATA=pickle.load(input)

#     nlp = spacy.blank('en')  # create blank Language class
#     # create the built-in pipeline components and add them to the pipeline
#     # nlp.create_pipe works for built-ins that are registered with spaCy
#     if 'ner' not in nlp.pipe_names:
#         ner = nlp.create_pipe('ner')
#         nlp.add_pipe(ner, last=True)

#     # add labels
#     for _, annotations in TRAIN_DATA:
#         for ent in annotations.get('entities'):
#             ner.add_label(ent[2])

#     # get names of other pipes to disable them during training
#     other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
#     with nlp.disable_pipes(*other_pipes):  # only train NER
#         optimizer = nlp.begin_training()
#         for itn in range(1):
#             print("Statring iteration " + str(itn))
#             random.shuffle(TRAIN_DATA)
#             losses = {}
#             for text, annotations in TRAIN_DATA:
#                 nlp.update(
#                     [text],  # batch of texts
#                     [annotations],  # batch of annotations
#                     drop=0.2,  # dropout - make it harder to memorise data
#                     sgd=optimizer,  # callable to update weights
#                     losses=losses)
#             print(losses)

In [6]:
# training_pickle_file = 'Multi_Train_Drug_data'

In [7]:
#read pickle file to load training data
# with open(training_pickle_file, 'rb') as input:
#     TRAIN_DATA=pickle.load(input)

FileNotFoundError: [Errno 2] No such file or directory: 'Multi_Train_Drug_data'

In [7]:
TRAIN_DATA

[('The dose should be prescribed using a combination of whole tablet strengths (62.5 mg, 125 mg, 250 mg, 375 mg).',
  {'entities': [(76, 81, 'Strength'),
    (82, 85, 'Strength'),
    (86, 89, 'Strength'),
    (90, 93, 'Strength'),
    (94, 97, 'Strength'),
    (98, 101, 'Strength'),
    (102, 105, 'Strength'),
    (106, 110, 'Strength')]}),
 ('Dogs   The recommended dosage of CLAVAMOX CHEWABLE Tablet is 6.25 mg/lb of body weight twice a day.',
  {'entities': [(61, 65, 'Dosage'),
    (66, 71, 'Dosage'),
    (87, 92, 'Frequency'),
    (93, 94, 'Frequency'),
    (95, 99, 'Frequency')]}),
 ('Administer CERENIA Tablets orally at a minimum dose of 2 mg/kg (0.9 mg/lb) body weight once daily for up to 5 consecutive days',
  {'entities': [(63, 67, 'Dosage'),
    (68, 74, 'Dosage'),
    (87, 91, 'Frequency'),
    (92, 97, 'Frequency')]}),
 ('CERENIA Injectable Solution is recommended at a dose of 1 mg/kg once daily.',
  {'entities': [(56, 57, 'Dosage'),
    (58, 63, 'Dosage'),
    (64, 68, 'Fre

#### Creating a blank Language model

In [8]:
nlp = spacy.blank('en')

In [9]:
doc = nlp("Dosages from 22 to 33 mg/kg (10 to 15 mg/lb) produce anesthesia that is suitable for diagnostic or minor surgical procedures that do not require skeletal muscle relaxation.")
print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

Entities= []


#### Creating a NER pipeline

In [10]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)

#### Adding labels

In [11]:
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

####  Get names of other pipes to disable them during training

In [12]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

#### Train only NER

In [13]:
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for itn in range(500):
        print("Statring iteration " + str(itn))
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.1,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

        doc = nlp("Nubeqa is an androgen inhibitor drug that helps in non-metastatic castration-resistant prostate cancer. Prostate cancer is also one form of disease that is treated with this Nubeqa drug. Usually people take 600 mg/day dosage. Usual side effects are fatigue asthenia and back pain.")
        print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))
        doc = nlp("The usual dosage of this injection is (1.2 to 3 mL/lb) of body weight.")
        print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))
        doc = nlp("The dosage of nicotin tablet is (1.8 to 3 mg/kg) of body weight.")
        print("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))
        
#     nlp.save('model.h5')

/Users/subashgandyer/opt/anaconda3/envs/testing/lib/python3.8/site-packages/spacy/language.py:635: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  proc.begin_training(
/Users/subashgandyer/opt/anaconda3/envs/testing/lib/python3.8/site-packages/spacy/language.py:635: UserWarning: [W034] Please install the package spacy-lookups-data in order to include the default lexeme normalization table for the language 'en'.
  proc.begin_training(


Statring iteration 0
{'ner': 212.31095018683118}
Entities= []
Entities= []
Entities= []
Statring iteration 1
{'ner': 126.2251306126418}
Entities= []
Entities= []
Entities= []
Statring iteration 2
{'ner': 109.64333077361681}
Entities= ['mg/day dosage. Usual side effects are fatigue asthenia and back_Dosage', 'pain_Strength']
Entities= []
Entities= ['mg/kg) of body_Dosage', 'weight_Strength']
Statring iteration 3
{'ner': 82.49499220222052}
Entities= ['600_Dosage', 'mg/day dosage. Usual side effects are fatigue asthenia and back pain._Dosage']
Entities= []
Entities= ['3_Dosage', 'mg/kg) of body weight._Dosage']
Statring iteration 4
{'ner': 70.05319704674285}
Entities= ['600_Dosage', 'mg/day_Dosage', 'dosage._Frequency']
Entities= ['3_Dosage', 'mL/lb)_Dosage']
Entities= ['(1.8_Dosage', 'to 3_Dosage', 'mg/kg)_Dosage']
Statring iteration 5
{'ner': 42.91345645525166}
Entities= ['600_Dosage', 'mg/_Dosage', 'day dosage. Usual side effects_Dosage']
Entities= []
Entities= []
Statring iteration 6


{'ner': 1.0945757202912574e-07}
Entities= ['600_Dosage', 'mg/_Dosage', 'day_Frequency']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 45
{'ner': 7.121874477091824e-07}
Entities= ['600_Dosage', 'mg/_Dosage', 'day_Frequency']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 46
{'ner': 1.2605051458152302e-06}
Entities= ['600_Dosage', 'mg/_Dosage', 'day_Frequency']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 47
{'ner': 2.0810911543564782e-06}
Entities= ['600_Dosage', 'mg/_Dosage', 'day_Frequency']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 48
{'ner': 2.558290046140175e-07}
Entities= ['600_Dosage', 'mg/_Dosage', 'day_Frequency']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 49
{'ner': 1.7510313219847017e-07}
Entities= ['600

{'ner': 2.9003162199313834e-08}
Entities= ['600_Dosage', 'mg/_Dosage', 'day_Frequency']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 88
{'ner': 1.4022708799203718e-07}
Entities= ['600_Dosage', 'mg/_Dosage', 'day_Frequency']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 89
{'ner': 1.0526303805594873e-08}
Entities= ['600_Dosage', 'mg/_Dosage', 'day_Frequency']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 90
{'ner': 1.4591365832636485e-06}
Entities= ['600_Dosage', 'mg/_Dosage', 'day_Frequency']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 91
{'ner': 1.7832090259178664e-08}
Entities= ['600_Dosage', 'mg/_Dosage', 'day_Frequency']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 92
{'ner': 2.8111047066062413e-08}
Entities= ['6

{'ner': 6.899958171540034e-09}
Entities= ['600_Dosage', 'mg/_Dosage', 'day_Frequency']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 132
{'ner': 2.440070740209781e-08}
Entities= ['600_Dosage', 'mg/_Dosage', 'day_Frequency']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 133
{'ner': 1.1358875421094409e-08}
Entities= ['600_Dosage', 'mg/_Dosage', 'day_Frequency']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 134
{'ner': 2.4775355282913364e-08}
Entities= ['600_Dosage', 'mg/_Dosage', 'day_Frequency']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 135
{'ner': 2.0258100312589114e-09}
Entities= ['600_Dosage', 'mg/_Dosage', 'day_Frequency']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 136
{'ner': 5.548577201302667e-09}
Entities= [

...

{'ner': 7.741905656493194e-10}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 315
{'ner': 1.4361962859203238e-10}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 316
{'ner': 7.667403998842256e-11}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 317
{'ner': 1.438158941553499e-11}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 318
{'ner': 2.212690300575438e-09}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 319
{'ner': 2.024695343242218e-10}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 't

{'ner': 3.7326324857704805e-11}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 362
{'ner': 4.0185010505295255e-10}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 363
{'ner': 4.7194467514397535e-11}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 364
{'ner': 1.9632523470354232e-10}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 365
{'ner': 5.454955662136623e-11}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 366
{'ner': 7.060325850789673e-11}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage',

{'ner': 8.141724450361332e-11}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 409
{'ner': 2.6621745792187777e-11}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 410
{'ner': 3.873990259310749e-11}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 411
{'ner': 8.334152209262392e-10}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 412
{'ner': 9.962402633293423e-12}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 413
{'ner': 4.300140687723917e-10}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 't

{'ner': 7.980021698143363e-10}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 456
{'ner': 1.9934017313493228e-10}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 457
{'ner': 2.260750789382869e-11}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 458
{'ner': 3.6526050089463716e-11}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 459
{'ner': 2.6024236785362956e-11}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 'to_Dosage', '3_Dosage', 'mg/kg)_Dosage']
Statring iteration 460
{'ner': 6.791635212897415e-11}
Entities= ['600_Dosage', 'mg/day_Dosage']
Entities= []
Entities= ['(1.8_Dosage', 

{'ner': 131.63481439695175}
Entities= ['600_Dosage', 'mg/day dosage. Usual side effects are fatigue asthenia and back pain._Dosage']
Entities= []
Entities= ['3_Dosage']
Statring iteration 493
{'ner': 141.32835415891478}
Entities= []
Entities= []
Entities= []
Statring iteration 494
{'ner': 166.80757874798059}
Entities= ['non_Dosage', 'take_Dosage', 'day_Dosage']
Entities= ['1.2_Dosage', 'to_Dosage']
Entities= ['(_Dosage', 'to_Dosage']
Statring iteration 495
{'ner': 92.15172453818792}
Entities= ['mg/day dosage. Usual side effects are fatigue asthenia and back pain._Dosage']
Entities= []
Entities= []
Statring iteration 496
{'ner': 68.98415987260424}
Entities= ['mg/day dosage._Dosage', 'pain._Frequency']
Entities= ['(1.2_Dosage', 'to 3_Dosage', 'weight._Frequency']
Entities= ['(1.8_Dosage', 'to 3_Dosage', 'mg/kg) of_Dosage', 'weight._Frequency']
Statring iteration 497
{'ner': 45.159398258155896}
Entities= []
Entities= []
Entities= ['(1.8_Dosage']
Statring iteration 498
{'ner': 34.277941967

### Serializing the built model

In [21]:
# One method - **Not used right now**
bytes_data = nlp.to_bytes()
lang = nlp.meta["lang"]  # "en"
pipeline = nlp.meta["pipeline"]  # ["tagger", "parser", "ner"]

In [22]:
# write to the disk
nlp.to_disk('./current_spacy_drugmodel_multi-letters_v2')

### Load the model

In [23]:
loaded_nlp = spacy.load("./current_spacy_drugmodel_multi-letters_v2")

### Test the model with a test document

In [24]:
doc = loaded_nlp("The usual dose of Furosemide Syrup 1 % is 1 to 2 mg/lb body weight (approximately 2.5 to 5 mg/kg).")
print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

Entities= ['2.5_Dosage', 'to_Dosage', '5_Dosage', 'mg/kg)._Dosage']


In [25]:
doc2 = loaded_nlp("Dogs—Administer intravenously or intramuscularly 0.25 to 0.50 mL per 10 pounds body weight once or twice daily at 6 to 8 hour intervals.")
print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc2.ents]))

Entities= ['0.25_Dosage', 'to_Dosage', '0.50_Dosage', 'mL_Dosage', 'per_Dosage', '10_Dosage', 'pounds_Dosage', 'body_Dosage', 'weight_Dosage', 'once_Frequency', 'or_Frequency', 'twice_Frequency', 'daily_Frequency']


In [26]:
doc3 = loaded_nlp("The dosage of some drug is 10 mg/kg twice a day.")
print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc3.ents]))

Entities= ['twice_Frequency', 'a_Frequency', 'day._Frequency']


In [27]:
doc4 = loaded_nlp("The strengths of some drug are 100 mg, 125 mg.")
print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc4.ents]))

Entities= ['mg,_Strength', '125_Strength', 'mg._Strength']


In [ ]:
Steps Database
- Train more with more samples [Optional]
- Take the XML
- Convert them into JSON
- Pass the text of Dosage & Administration Section into this NER model
- Extract the entities (Dosage, Frequency, Strength, and so on)
- Repeat this for all XML files (3000+)

Clean Steps
- Clean the database

API Steps
- Wrap it around the database
- Provide the data in a JSON format

Deploy Steps
- GCP
- Test endpoints

### Testing other Vitrakvi documents

In [80]:
doc = loaded_nlp("""Pharmacokinetics of VITRAKVI® (larotrectinib)

FDA Approved Indication 
VITRAKVI is indicated for the treatment of adult and pediatric patients with solid tumors that: 
have a neurotrophic receptor tyrosine kinase (NTRK) gene fusion without a known acquired resistance mutation,
are metastatic or where surgical resection is likely to result in severe morbidity, and
have no satisfactory alternative treatments or that have progressed following treatment.
This indication is approved under accelerated approval based on overall response rate and duration of response. Continued approval for this indication may be contingent upon verification and description of clinical benefit in confirmatory trials.

Summary1
The pharmacokinetics of larotrectinib were studied in healthy subjects and adult and pediatric patients with locally advanced or metastatic solid tumors.
In adult patients who received VITRAKVI capsules 100 mg twice daily in Study LOXO-TRK-14001, peak plasma levels (Cmax) of larotrectinib were achieved at approximately 1 hour after dosing and steady state was reached within 3 days. 
The mean absolute bioavailability of VITRAKVI capsules was 34% (range: 32% to 37%). In healthy subjects, the AUC of VITRAKVI oral solution was similar to that of the capsules and the Cmax was 36% higher with the oral solution.
Larotrectinib is metabolized predominantly by CYP3A4.
Following oral administration of a single [14C] radiolabeled 100 mg dose of larotrectinib to healthy subjects, 58% (5% unchanged) of the administered radioactivity was recovered in feces and 39% (20% unchanged) was recovered in urine. 
In healthy subjects who received a single dose of VITRAKVI capsules, systemic exposure (Cmax and AUC) of larotrectinib was dose proportional over the dose range of 100 mg to 400 mg (1 to 4 times the recommended adult dose) and slightly greater than proportional at doses of 600 mg to 900 mg (6 to 9 times the recommended adult dose).

Prescribing Information1
The pharmacokinetics of larotrectinib were studied in healthy subjects and adult and pediatric patients with locally advanced or metastatic solid tumors.
In healthy subjects who received a single dose of VITRAKVI capsules, systemic exposure (Cmax and AUC) of larotrectinib was dose proportional over the dose range of 100 mg to 400 mg (1 to 4 times the recommended adult dose) and slightly greater than proportional at doses of 600 mg to 900 mg (6 to 9 times the recommended adult dose). In adult patients who received VITRAKVI capsules 100 mg twice daily in Study LOXO-TRK-14001, peak plasma levels (Cmax) of larotrectinib were achieved at approximately 1 hour after dosing and steady-state was reached within 3 days. Mean steady-state larotrectinib [coefficient of variation (CV%)] for Cmax was 788 (81%) ng/mL and AUC0-24hr was 4351 (97%) ng*h/mL.

Absorption
The mean absolute bioavailability of VITRAKVI capsules was 34% (range: 32% to 37%). In healthy subjects, the AUC of VITRAKVI oral solution was similar to that of the capsules and the Cmax was 36% higher with the oral solution.
Effect of Food
The AUC of larotrectinib was similar and the Cmax was reduced by 35% after oral administration of a single 100 mg capsule of VITRAKVI to healthy subjects taken with a high-fat meal (approximately 900 calories, 58 grams carbohydrate, 56 grams fat and 43 grams protein) compared to the Cmax and AUC in the fasted state.
Distribution1
The mean (CV%) volume of distribution (Vss) of larotrectinib is 48 (38%) L following intravenous administration of larotrectinib in healthy subjects. Larotrectinib is 70% bound to human plasma proteins in vitro and binding is independent of drug concentrations. The blood-to-plasma concentration ratio is 0.9.
Elimination1
The mean (CV%) clearance (CL/F) of larotrectinib is 98 (44%) L/h and the half-life is 2.9 hours following oral administration of VITRAKVI in healthy subjects. 
Metabolism
Larotrectinib is metabolized predominantly by CYP3A4. Following oral administration of a single [14C] radiolabeled 100 mg dose of larotrectinib to healthy subjects, unchanged larotrectinib constituted 19% and an O-linked glucuronide constituted 26% of the major circulating radioactive drug components in plasma. 
Excretion
Following oral administration of a single [14C] radiolabeled 100 mg dose of larotrectinib to healthy subjects, 58% (5% unchanged) of the administered radioactivity was recovered in feces and 39% (20% unchanged) was recovered in urine. 
Specific Populations1
Age (range: 28 days to 82 years), sex, and body weight (range: 3.8 to 179 kg) had no clinically meaningful effect on the pharmacokinetics of larotrectinib. 
Pediatric Patients
In pediatric patients, the larotrectinib geometric mean (%CV) AUC0-24hr by age subgroup was: 3348 (66%) ng*h/mL in patients 1 month to < 2 years (n = 9), 4135 (36%) ng*h/mL in patients 2 to < 12 years (n = 15), and 3108 (69%) ng*h/mL and in patients 12 to < 18 years (n = 9). 
Patients with Renal Impairment 
Following oral administration of a single 100 mg dose of VITRAKVI capsules in subjects with end-stage renal disease (e.g., subjects who required dialysis), the AUC0-inf of larotrectinib increased 1.5-fold and Cmax increased 1.3-fold as compared to that in subjects with normal renal function (creatinine clearance ≥ 90 mL/min as estimated by Cockcroft-Gault). The pharmacokinetics of VITRAKVI in patients with moderate to severe renal impairment (creatinine clearance ≤ 60 mL/min) have not been studied. 

Patients with Hepatic Impairment 
Following oral administration of a single 100 mg dose of VITRAKVI capsules, the AUC0-INF of larotrectinib increased 1.3-fold in subjects with mild hepatic impairment (Child-Pugh A), 2-fold in subjects with moderate hepatic impairment (Child-Pugh B) and 3.2-fold in subjects with severe hepatic impairment (Child-Pugh C) as compared to that in subjects with normal hepatic function. The Cmax was similar in subjects with mild and moderate hepatic impairment and the Cmax of larotrectinib increased 1.5-fold in subjects with severe hepatic impairment as compared to that in subjects with normal hepatic function.
Drug Interaction Studies1
Clinical Studies 
Effect of Strong CYP3A Inhibitors: Coadministration of a single 100 mg dose of VITRAKVI capsules with a strong CYP3A inhibitor (itraconazole) increased the AUC0-INF of larotrectinib by 4.3-fold and the Cmax by 2.8-fold as compared to VITRAKVI administered alone. The effects of CYP3A moderate and weak inhibitors on the pharmacokinetics of larotrectinib have not been studied. 
Effect of Strong CYP3A Inducers: Coadministration of a single 100 mg dose of VITRAKVI capsules with a strong CYP3A inducer (rifampin) decreased the AUC0-INF of larotrectinib by 81% and of Cmax by 71% as compared to VITRAKVI administered alone. The effects of CYP3A weak and moderate inducers on the pharmacokinetics of larotrectinib have not been studied. 
Effect of Strong P-glycoprotein (P-gp) Inhibitors: Coadministration of a single 100 mg dose of VITRAKVI capsules with a P-gp inhibitor (rifampin) increased the AUC0-INF of larotrectinib by 1.7-fold and the Cmax by 1.8-fold as compared to VITRAKVI administered alone. 
Effect of Larotrectinib on CYP3A4 Substrates: Coadministration of VITRAKVI capsules 100 mg twice daily with a sensitive CYP3A4 substrate (midazolam) increased both the AUC0-INF and Cmax of midazolam by 1.7-fold as compared to midazolam administered alone. The AUC0-INF and Cmax of 1-hydroxymidazolam, the main metabolite of midazolam, were both increased 1.4-fold as compared to when midazolam was administered alone.
In Vitro Studies
Effect of Transporter on Larotrectinib: Larotrectinib is a substrate for P-gp and BCRP. Larotrectinib is not a substrate of OAT1, OAT3, OCT1, OCT2, OATP1B1, or OATP1B3.
Effect of Larotrectinib on Transporters: Larotrectinib is not an inhibitor of BCRP, P-gp, OAT1, OAT3, OCT1, OCT2, OATP1B1, OATP1B3, BSEP, MATE1 and MATE2-K at clinically relevant concentrations.
Effect of Larotrectinib on CYP Substrates: Larotrectinib is not an inhibitor or inducer of CYP1A2, CYP2B6, CYP2C8, CYP2C9, CYP2C19, or CYP2D6 at clinically relevant concentrations.


Published Literature2-4

Pharmacokinetics in Adults2,3 

A Phase 1, multicenter, open-label, 3+3 dose-escalation and safety 1 study was conducted to evaluate the safety, maximum-tolerated dose, and pharmacokinetic parameters of larotrectinib in adults with advanced solid tumors. Larotrectinib was administered for a single day between 3 to 7 days before the start of Cycle 1, in which subjects received QD or BID doses for continuous 28-day cycles.2-3 Plasma concentrations were quantified by a validated LC-MS/MS assay after the single-day dose, and on days 1 and 8 of Cycle 1.3 Table 1 contains the pharmacokinetic parameters of larotrectinib at different dose levels. The concentration-time curve for days 1 and 8 of cycle 1 is shown in Figure 1.

Figure 1a: The horizontal line representing TRKA IC50 refers to the total plasma concentration of larotrectinib that is associated with an unbound concentration of larotrectinib that is equal to its IC50 for inhibition of NGF-stimulated activity in a cellular assay. The TRKA IC90 is modeled from IC50 data. The IC50 and IC90 values for TRKB and TRKC are not shown, but are similar to those of TRKA.

Figure 1b: AUC data with additional dose schedules. Dotted lines in the right panel are inferred pharmacokinetics from the evening BID dose

Figure 1c: Cmax of larotrectinib. The horizontal line representing TRKA IC90 refers to the total plasma concentration of larotrectinib that is associated with an unbound concentration of larotrectinib that is equal to its IC90 for inhibition of NGF-stimulated activity in a cellular assay. The IC90 values for TRKB and TRKC are not shown, but are similar to those of TRKA. 

Pharmacokinetics in Pediatric Patients4 

A multicenter, open-label Phase 1/2 study was conducted to evaluate the safety including dose-limiting toxicities (DLT), maximum tolerated dose (MTD), pharmacokinetics and efficacy of oral larotrectinib in pediatric patients.4  

The larotrectinib AUC and Cmax increased proportionally with dose per body surface area (BSA). There was no apparent difference in Cmax or AUC in patients treated with capsule versus liquid formulation or among different pediatric age groups. The concentration-time curve for adults with 100mg BID and pediatric with capsule and liquid formulation (100mg/m2 BID, maximum 100mg per dose) is shown in Figure 3.  Figure 4 shows AUC data by age groups.4 

Figure 2: Concentration-Time Curve (Adults vs. Pediatric)

Figure 3: AUC data by Age Groups

""")

In [82]:
colors = {"DRUG":"#ff9ff0",
         "DISEASE": "turquoise",
         "ADVERSE EVENT": "tomato",
         "CONDITION": "lavender",
         "DOSAGE": "#ffa500",
         "INDICATION": "#ffe6ff",
         "PHARMACOKINETICS": "#b3b3cc",
         "DOSING": "#ffd9b3",
         "EFFICACY": "#d9ffb3",
         "CONTRAINDICATIONS": "#ffa366",
         "SAFETY": "#80ff80",
         "DRUG INTERACTION": "#b3d9ff",
         "FOOD INTERACTION": "#3399ff",
         "BIOAVAILABILITY": "#ff9900",
         "ABSORPTION": "#ccccb3",
         "AUC": "#00ff55",
         "CMAX": "#ff531a",
         "MFS": "#d966ff",
         "OS": "#5c5cd6",
         "TTPP": "#0088cc",
         "PFS": "#ffcccc",
         "QOL": "#cc00cc",
         "OBJECTIVE": "#ffffb3",
         "PLASMA CONCENTRATION": "#ffff1a",
         "DOSAGE FORM": "#f2f2f2",
         "TMAX": "#b3ffff",
         "ENDPOINT": "#ccddff",
         "DISCONTINUATION": "#ccffcc",
         "DEATH": "#ccffe6",
         "ADMINISTRATION": "#ff4da6"}
options = {"ents": ["DRUG", "DISEASE", "ADVERSE EVENT", "CONDITION", "DOSAGE", "INDICATION", "PHARMACOKINETICS", "DOSING", "EFFICACY", "CONTRAINDICATIONS", "SAFETY", "DRUG INTERACTION", "FOOD INTERACTION", "BIOAVAILABILITY", "ABSORPTION", "AUC", "CMAX", "MFS", "OS", "TTPP", "PFS", "QOL", "OBJECTIVE", "PLASMA CONCENTRATION", "DOSAGE FORM", "TMAX", "ENDPOINT", "DISCONTINUATION", "DEATH", "ADMINISTRATION"], "colors": colors}
displacy.serve(doc, style="ent", options=options)

/Users/subashgandyer/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


#### Testing the built NER model

In [11]:
doc = nlp("Nubeqa is an androgen inhibitor drug that helps in non-metastatic castration-resistant prostate cancer. Prostate cancer is also one form of disease that is treated with this Nubeqa drug. Usually people take 600 mg/day dosage. Usual side effects are fatigue asthenia and back pain.")
print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

Entities= ['Nubeqa_Drug', 'non-metastatic castration-resistant prostate cancer_Disease', 'Nubeqa_Drug', '600 mg/day_Dosage', 'asthenia_Drug', 'back_Condition']


In [12]:
displacy.render(doc, jupyter=True, style='ent')

In [13]:
doc = nlp("Nubeqa is an androgen inhibitor drug that helps in non-metastatic castration-resistant prostate cancer. Usually people take 600 mg/day dosage. Usual side effects are fatigue and back pain.")
print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

Entities= ['Nubeqa_Drug', 'non-metastatic castration-resistant prostate cancer_Disease', '600 mg/day_Dosage', 'fatigue_Adverse Event', 'back_Condition']


In [14]:
displacy.render(doc, jupyter=True, style='ent')

In [15]:
doc = nlp("Nubeqa helps in castration-resistant prostate cancer. It is to be administered as a 600 mg/day dosage. Usually fatigue and back pain are the side effects for this drug.")
print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

Entities= ['Nubeqa_Drug', '600 mg/day_Dosage', 'back_Adverse Event']


In [16]:
displacy.render(doc, jupyter=True, style='ent')

In [17]:
doc = nlp("Nubeqa")
print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

Entities= ['Nubeqa_Drug']


In [18]:
displacy.render(doc, jupyter=True, style='ent')

In [19]:
doc = nlp("non-metastatic castration-resistant prostate cancer")
print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

Entities= ['non-metastatic castration-resistant prostate cancer_Disease']


In [20]:
displacy.render(doc, jupyter=True, style='ent')

In [21]:
doc = nlp("600 mg/day dosage.")
print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

Entities= ['600 mg_Strength', 'day_Condition']


In [22]:
displacy.render(doc, jupyter=True, style='ent')

In [23]:
doc = nlp("fatigue and back pain are the side effects for this drug.")
print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

Entities= ['fatigue_Adverse Event', 'back_Adverse Event']


In [24]:
displacy.render(doc, jupyter=True, style='ent')

#### Wrapping all above steps into a python script

In [ ]:
def train_spacy(TRAIN_DATA):
    # TRAIN_DATA = convert_dataturks_to_spacy(filename);
    # TRAIN_DATA = convert_dataturks_to_spacy(data)
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(1000):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.1,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)

            doc = nlp("Nubeqa is an androgen inhibitor drug that helps in non-metastatic castration-resistant prostate cancer. Prostate cancer is also one form of disease that is treated with this Nubeqa drug. Usually people take 600 mg/day dosage. Usual side effects are fatigue asthenia and back pain.")
            print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))
            doc = nlp("Nubeqa is an androgen inhibitor drug that helps in non-metastatic castration-resistant prostate cancer. Usually people take 600 mg/day dosage. Usual side effects are fatigue and back pain.")
            print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

    
    #do prediction
    doc = nlp("Nubeqa is an androgen inhibitor drug that helps in non-metastatic castration-resistant prostate cancer. Prostate cancer is also one form of disease that is treated with this Nubeqa drug. Usually people take 600 mg/day dosage. Usual side effects are fatigue asthenia and back pain.")
    print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

    doc = nlp("Nubeqa is an androgen inhibitor drug that helps in non-metastatic castration-resistant prostate cancer. Usually people take 600 mg/day dosage. Usual side effects are fatigue and back pain.")
    print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

In [ ]:
training_data = convert_dataturks_to_spacy(data)
training_data

In [ ]:
train_spacy(training_data)

### Entity Extraction Demo from Saved Trained Model

In [83]:
import spacy
from spacy import displacy
import random
import json
import logging

In [84]:
# loaded_nlp = spacy.load("./current_spacy_model")
loaded_nlp = spacy.load("./current_spacy_model_multi-letters")

In [85]:
doc = loaded_nlp("Nubeqa is an androgen inhibitor drug that helps in non-metastatic castration-resistant prostate cancer. Prostate cancer is also one form of disease that is treated with this Nubeqa drug. Usually people take 600 mg/day dosage. Usual side effects are fatigue asthenia and back pain.")
print ("Entities= " + str(["" + str(ent.text) + "_" + str(ent.label_) for ent in doc.ents]))

Entities= ['Nubeqa_Drug', 'androgen inhibitor_Drug', 'non-metastatic castration-resistant prostate cancer_Condition', 'Nubeqa_Drug', '600 mg_Strength', 'fatigue_Adverse Event', 'asthenia_Adverse Event', 'back pain_Adverse Event']


In [86]:
colors = {"DRUG1": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
          "DRUG":"#ff9ff0",
         "DISEASE": "turquoise",
         "ADVERSE EVENT": "tomato",
         "CONDITION": "lavender",
         "DOSAGE": "#ffa500"}
options = {"ents": ["DRUG", "DISEASE", "ADVERSE EVENT", "CONDITION", "DOSAGE"], "colors": colors}
displacy.serve(doc, style="ent", options=options)

/Users/subashgandyer/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


## Testing other out of sample documents

### Document 1 - Nubeqa (US-NUBEQA-76043-S.docx-3.pdf)

In [103]:
doc = loaded_nlp("""Clinical Data on NUBEQA®   (darolutamide) in Patients with Progressive Metastatic Castrate Resistant Prostate Cancer
Summary:
● Nubeqa (darolutamide) is an androgen receptor inhibitor indicated for the treatment of patients with non-metastatic castration-resistant prostate cancer.1 
● Nubeqa is not approved for use in patients with progressive metastatic castrate resistant prostate cancer.
● Phase 1–2 clinical trial was conducted to assess safety and tolerability of darolutamide in men with progressive metastatic castration-resistant prostate cancer.2    Secondary objectives were to determine the  pharmacokinetics , and  antitumor activity  as defined by changes in serum prostate-specific antigen (PSA), by imaging of soft tissue and bone lesions, and by changes in circulating tumor cell counts.2 
o An open-label, nonrandomized phase 1 dose escalation portion and a phase 2 portion included randomized dose expansion and long-term follow-up.
o Twenty-four patients were enrolled in the   dose escalation phase. No intra-patient dose escalation was permitted. Patients received 200 mg/day (n=4); 400 mg/day (n=7); 600 mg/day (n=3); 1000 mg/day (n=4); 1400 mg/day (n=3); 1800 mg/day (n=3).
o In the phase 2 part of the study (n=112):
▪ Thirty-eight patients were assigned to receive 200 mg/day (11
chemotherapy naïve and CYP17inhibitor (CYP17i); naïve 9 post
chemotherapy and CYP17i naïve; 18 post CYPi).
▪ Thirty-seven patients were   assigned to receive 400 mg/day (9
[chemotherapy naïve and CYP17i) naïve]; 9 [post chemotherapy and
CYP17i naïve]; 19 [post CYPi]).
▪ Thirty-five patients were   assigned to receive 1400 mg/day (8
[chemotherapy naïve and CYP17i) naïve]; 10 [post chemotherapy and
CYP17i naïve]; 17 [post CYPi]).
▪ Pharmacokinetic analysis darolutamide showed rapid absorption and the
median t max  ranged 3·0–5·1 hours for darolutamide and 1.5–5.0 h for the diastereo isomer keto-darolutamide on day 1. The mean metabolite to parent ratio (area under the curve, AUC) at steady state was 1.6–2.3:1. The exposure of darolutamide (AUC t a  nd C max)  at steady state increased in a linear, dose-related manner up to 1400 mg daily dose.
o Most common Grade 3 adverse events were fatigue or asthenia 2 (2%), back pain 2 (2%) and arthralgia 2 (2%).
o PK response (≥ 50% decrease in PSA) by week 12 was observed with all doses and all treatment groups, but significantly fewer were seen in patients previously treated with CYP17 inhibitors than in those who were naïve to both chemotherapy and CYP17 inhibitors.
o RECIST responses or stable disease in soft tissue, and stabilized disease in bone was observed at all dose levels. The highest activity was noted in patients who were chemotherapy-naive and CYP17 inhibitor-naive patients
NUBEQA®  (  darolutamide) Clinical Data (ARADES & ARAFOR)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
 o Circulating tumor cell measured at baseline and at week 12 (available in 87 of 124 patients) 41 (82%) patients maintained favorable circulating tumor cell counts (<5 cells per 7·5 mL of blood) at week 12. In the patients with ≥5 cells counts at baseline, 14 (38%) converted to < 5 cells counts, and nine (18%) patients converted from < 5 cells counts to ≥5 cells counts.
o Patients (58%) who were both chemotherapy and CYP17 inhibitor-naïve converted from ≥5 cells counts to < 5 cells counts.
o The median time to PSA progression in phase 1–2 (including all dose levels) was 72.3 weeks (95% CI 24.3–not reached [NR]) for patients who were naive for both chemotherapy and CYP17 inhibitor.
o Median  time to radiological progression  was not reached for patients who were naive to both chemotherapy and CYP17 inhibitor (95% CI 36.4–NR).
● An open randomized phase 1 trial ( ARAFOR ) evaluated  bioavailability  and  effect of food on pharmacokinetics of darolutamide.3    The study compared relative bioavailability of 600 mg tablets (tablet A and tablet B) and capsule (reference) formulations in patients with chemotherapy naïve metastatic castrate resistant prostate cancer.  The study also assessed  the effect of food on the bioavailability of darolutamide tablet products tablet A (TabA) and tablet B (TabB). The study also evaluated short term safety and tolerability and in an extension component long-term safety and tolerability, which was ongoing at the time of presentation.3 
o Patients were randomized to one of 3 treatment sequences in each group .  There were three treatment periods for each patient during the PK Component which included administration of two darolutamide 300 mg test tablets (TabA or TabB) in both fasted and fed conditions, or administration of darolutamide 100 mg capsule at 600 mg dose in fed state. There was at least a 7 day wash-out period between the three study treatment administrations. The total daily dose was 600 mg/day
o Thirty men with metastatic chemotherapy naïve CRPC were enrolled. Area under the curve (AUC) and maximum concentration (C max)  values were approximately two times lower after administration at fast. With high fat meal the C max  occurred about 2-3 hours later. The  food effect  was similar in all groups.
o The pharmacokinetics observed with the two new  tablet  formulations were comparable with the pharmacokinetics of the  capsule  formulation.
o The PSA response rate (≥50% PSA decline) was 87% (26/30), with a median PSA decrease of 71% (-96, 5) at week 4
o Most commonly reported adverse events (>10%) were fatigue/asthenia (13%), nausea (10%), erythema (10%) and urinary tract disorder (10%); all being mild events except one nausea event (grade 3).
Nubeqa (darolutamide) is an androgen receptor inhibitor indicated for the treatment of patients with non-metastatic castration-resistant prostate cancer.1 
Nubeqa is not approved for use in patients with progressive metastatic castrate resistant prostate cancer.
A Phase 1–2 clinical trial was conducted to assess darolutamide in men with progressive metastatic castration-resistant prostate cancer.2    The primary objectives of the trial were to
NUBEQA®  (  darolutamide) Clinical Data (ARADES & ARAFOR)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
       
 assess safety and tolerability and to define a maximum tolerated dose. Secondary objectives were to determine the pharmacokinetics, and antitumor activity as defined by changes in serum prostate-specific antigen (PSA), by  imaging of soft tissue and bone lesions , and by changes in circulating tumor cell counts.
In an open-label, multicenter trial with a nonrandomized phase 1 dose escalation portion, a phase 2 portion included randomized dose expansion and long-term follow-up.2    Patients with adenocarcinoma of the prostate with progressive metastatic disease despite ongoing androgen-deprivation therapy were enrolled.
The phase 1 dose-escalation part of the study was not randomized.1    The patients sequentially entered the phase 1 part of the study. In the phase 2 portion, patients were randomly assigned (ratio 1:1) by blocks of four and two to 200 mg or 400 mg daily doses of darolutamide with previous treatment (chemotherapy-naive and CYP17 inhibitor-naive; post-chemotherapy and CYP17 inhibitor-naive; and post- CYP17 inhibitor) as stratification factors. The third expanded dose level, patients were randomly assigned into 200 mg, 400 mg, or 1400 mg daily doses.
Three to six patients sequentially received 200 mg (100 mg twice daily) to 400 mg, 600 mg, 1000 mg, 1400 mg, or 1800 mg.2    In the absence of grade 3 or higher treatment-related adverse events within the first 28 days patients were permitted to continue darolutamide treatment in the absence of disease progression.2    No intra-patient dose escalation was permitted.
The phase 2 part of the study included three expanded cohorts (200 mg, 400 mg, and 1400 mg daily dose) each containing about 35 patients.2    A total of 136 patients were enrolled (24 were enrolled in the dose-escalation phase and 112 randomly assigned to receive either 200 mg, 400 mg, or 1400 mg of darolutamide). Of the 24 patients enrolled in dose escalation phase; 200 mg/day (n=4, 1 discontinued protocol violation); 400 mg/day (n=7, 1 discontinued due to AE); 600 mg/day (n=3, 0 discontinued); 1000 mg/day (n=4, 1 discontinued due to AE); 1400 mg/day (n=3, 0 discontinued); 1800 mg/day (n=3, 0 discontinued). Allocation of the 112 patients randomized in the phase 2 part of the study is presented in Figure 1. Two patients randomized did not receive darolutamide.
 NUBEQA®  (  darolutamide) Clinical Data (ARADES & ARAFOR)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 Figure 1: Trial profile of the phase 2 part of the study1 
 Select baseline characteristic of patients are presented in Table 1.2  Table 1: Select Baseline Characteristics2 
        Phase1 (n=24)
 Phase 2 Group 1
     Group 2
  Group 3
          200mg (n=38)
      400mg (n=37)
   1400 (n=35)
   Age (years)
      69 (64–74)
 67 (65–73)
      68 (64–75)
   73 (64–78)
   PSA (ng/ml)
      27 (13–96)
 109 (43–352)
      95 (45–255)
   155 (31–382)
 LDH (U/L)
  249 (192–412)
   220 (195–292)
    232 (202–322)
    226 (201–322)
   Chemotherapy-naive and CYP17 inhibitor-naive
    14 (58
11 (29%)
    9 (24%)
 8 (23%)
   Post-chemotherapy and CYP17 inhibitor-naïve
      7 (29%)
 9 (24%)
      9 (24%)
   10 (29%)
 Post-CYP17 inhibitor
  3 (13%)
   18 (47%)
    19 (51%)
    17 (49%)
   Total Gleason Score <8
> 8
 15 (68%) 7 (32%)
15 (42%) 21 (58%)
  20 (59%) 14 (41%)
 15 (46%) 18 (55%)
  NUBEQA®  (  darolutamide) Clinical Data (ARADES & ARAFOR)
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

   Missing
 2
  2
   3
   2
   ECOG PS 0
1
    17 (71%) 7 (29%)
19 (50%) 19 (50%)
    20 (54%) 17 (46%)
 16 (46%) 19 (54%)
 Circulating tumor cell counts <5 cells/7.5mL of blood
> 5 cells/7.5mL of blood
   13 (54%) 9 (38%)
   18 (47%) 17 (45%)
    18 (49%) 17 (46%)
    16 (46%) 14 (40%)
   Disease localization Bone disease
Bone only
Bone and soft tissue Soft-tissue disease Lymph node Visceral
No radiologic evidence of mets
    15 (63%) 6 (25%) 9 (38%) 17 (71%) 17 (71%) 1 (4%) 1 (4%)
34 (89%) 12 (32%) 22 (58%) 26 (68%) 18 (47%) 10 (26%) 0
    32 (86%) 11 (30%) 21 (57%) 26 (70%) 24 (65%) 8 (22%) 0
 32 (91%) 20 (57%) 12 (34%) 15 (43%) 7 (20%) 8 (23%) 0
 Bone metastases 0
1-4 5-20 >20
  9 (38%) 3 (13%) 7 (29%) 5 (21%)
  6 (16%) 10 (26%) 11 (29%) 11 (29%)
   5 (14%) 13 (35%) 9 (24%) 10 (27%)
   3 (9%) 6 (17%) 8 (23%) 18 (51%)
  PSA=prostate-specific antigen. LDH=lactate dehydrogenase. ECOG=Eastern Cooperative Oncology Group. NA=not assessable, Mets=metastases
The median time on darolutamide therapy was 24-8 (IQR 24.8-25.2) months for the phase 1 part of the study and 11 months (IQR 8.3-14.0) months for phase 2 part of the trial.1    Most common reason for discontinuation was disease progression and 5 (4%) of 124 patients discontinued for adverse events.
Common adverse events occurring in greater than equal to 10% of patients are presented in Table 2 below.2 
Table 2: Most Common Adverse Events Occurring in  >  10% of Patients2 
        Grade 1
   Grade 2
  Grade 3
   Fatigue or asthenia
      27 (22%)
    13 (10%)
   2 (2%)
   Back pain
      16 (13%)
    13 (10%)
   2 (2%)
   Arthralgia
      15 (12%)
    9 (7%)
   2 (2%)
   Constipation
      14 (11%)
    5 (4%)
   0
   Decreased appetite
      15 (12%)
    1 (<1%)
   0
   Nausea
      12 (10%)
    4 (3%)
   0
 Peripheral edema
  13 (10%)
    2 (2%)
    0
 Pharmacokinetic analysis darolutamide showed rapid absorbed and the median t max  were 3·0–5·1 h for darolutamide and 1·5–5·0 hours for keto-darolutamide on day 1.2    The mean metabolite to parent ratio (area under the curve, AUC) at steady state was 1·6–2·3:1. The exposure of darolutamide (AUC t  and C max)  at steady state increased in a linear, dose-related manner up to 1400 mg daily dose.
NUBEQA®  (  darolutamide) Clinical Data (ARADES & ARAFOR)
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 Antitumor activity as measured by in PSA from baseline are presented in Figure 2.
Figure 2: Percentage in PSA Decline at Week 12 From Baseline2 
 RECIST responses or stable disease in soft tissue, and stabilized disease in bone are shown in table 4.2    No differences in response by dose were noted in soft tissue and bone. This variable was not formally evaluated due to the small number of patients within each dose group and subgroup. The highest activity was noted in patients who were chemotherapy-naive and CYP17 inhibitor-naive patients (table 3).
Table 3: Imaging Results at Week 12 Using RECIST2 
NUBEQA ® (  darolutamide) Clinical Data (ARADES & ARAFOR)
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

  Measured baseline and 12-week circulating tumor cell results were available in 87 of 124 patients.2    Circulating conversion rates of circulating tumor cells are presented in table 4 below.
Table 4: Circulating Tumor Cells at Week 12: Expanded dose Levels2 
        Chemotherapy naïve/CYP17i naïve (n=37*)
 Post Chemotherapy/CYP17i naïve
(n=32*)
    Post CYP 17i (n=55*)
     < at Baseline
    (n=18)
 (n=16)
     (n=25)
     <5 at week 12
    15 (88%)
 10 (91%)
     16 (73%)
     ≥5 at week 12
    2 (12%)
 1 (9%)
     6 (27%)
     NA at week 12
    1
 5
     3
     ≥5 at baseline
    (n=16)
 (n=13)
     (n=24)
     <5 at week 12
    7 (58%)
 7 (58%)
     0
 ≥5 at week 12
    5 (42%)
   ,5 (42%)
   1213 (100%)
  NUBEQA ® (  darolutamide) Clinical Data (ARADES & ARAFOR)
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

  NA at week 12
   4
  1
  11
NA=not assessable. CTC count is expressed as cells per 7·5 mL of blood. These data include patients with missing baseline CTC. *Total number includes three patients with missing baseline. †Total number includes six patients with missing baseline.
The median serum testosterone level at baseline was 0·4 nmol/L (IQR 0·4–0·6; n=124) and castrate concentrations were maintained at week 12 (0·4 nmol/L [0·4–0·6]; n=111).2 
The median time to PSA progression in phase 1–2 (including all dose levels) was 72·3 weeks (95% CI 24·3–not reached [NR]) for patients who were naive to both chemotherapy and CYP17 inhibitor; 20·3 weeks (95% CI 16·9–26·1) for those who had received chemotherapy but not a CYP17 inhibitor, and 19·3 weeks (95% CI 14·1–27·1) for those who had received a CYP17 inhibitor (figure 3).2 
Figure 3: Time to PSA Progression2 
Median time to radiological progression was not reached for patients who were naive to both chemotherapy and CYP17 inhibitor (95% CI 36·4–NR); 50·4 weeks (95% CI 12·1–NR) for those who had received chemotherapy but not a CYP17 inhibitor; and 12·7 weeks (95% CI 12·0–36·3) for those who had received a CYP17 inhibitor (figure 4).2 
Figure 4: Time to Radiologic Progression2 
NUBEQA ® (  darolutamide) Clinical Data (ARADES & ARAFOR)
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
 
  Highest pre-specified daily dose of 1800 mg and a maximum tolerated dose was not reached; no dose-limiting toxic effects were noted and dose escalation was discontinued because of a plasma concentration plateau of darolutamide.
An open randomized phase 1 trial (ARAFOR) evaluated bioavailability and effect of food on bioavailability of  darolutamide .3    The study compared relative bioavailability of 600 mg tablets (tablet A and tablet B) and capsule (reference) formulations in patients with chemotherapy naïve metastatic castrate resistant prostate cancer. The study also assessed the effect of food on the bioavailability of  darolutamide  tablet products tablet A (TabA) and tablet B (TabB). The study also evaluated short term safety and tolerability and in an extension component long-term safety and tolerability (was ongoing at the time of presentation)
The PK Component consisted of two concurrent groups, which evaluated TabA and (TabB), and capsule.3    Patients were randomized to one of 3 treatment sequences in each group.3     There were three treatment periods for each patient during the PK Component which included administration of two  darolutamide  300 mg test tablet (TabA or TabB) in both fasted and fed conditions, or administration of  darolutamide  100 mg capsule at 600 mg dose in fed state. There was at least 7 days wash-out period between the three study treatment administrations. The total daily dose was 600 mg/day.
Thirty  men with metastatic chemotherapy-naïve CRPC were enrolled; the median age of the patients was 68 years (Table 5).3 
NUBEQA ® (  darolutamide) Clinical Data (ARADES & ARAFOR)
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 Table 5: Select Baseline Characteristics3 
   Characteristic
       Age Median (range
     68 (54-86)
 Baseline PSA (ng/mL)
  18.2 (4-555)
 Disease localization
Bone only
Soft tissue only
Both bone and soft tissue
   14 (46.7%) 3 (10%) 13 (43.3%)
   LHRH
     30 (100%
   Previous antiandrogen therapy
     22 (73.3%)
 Median Gleason score
  7.0
PSA = prostate specific antigen LHRH=Leutinizing hormone releasing hormone
All thirty patients completed the 4-week visit.3    Up to cut-off point 29/30 patients are continuing and 20 patients have completed 12 weeks in the trial so far. The PSA response rate (≥ 50% PSA decline) was 87% (26/30), with a median PSA decrease of -71% (-96, 5) at week 4 (Figure 5).
NUBEQA ® (  darolutamide) Clinical Data (ARADES & ARAFOR)
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 Figure 5: Maximum Change (%) in PSA during Trial3 
Data on PSA at 12 weeks were available in 20 patients and are presented in Figure 6.3  Figure 6: Change in PSA (%) at 12 Weeks2 
Darolutamide  was well tolerated. Most commonly reported adverse events (>10%) were fatigue/asthenia (13%), nausea (10%), erythema (10%) and urinary tract disorder (10%); all being mild events except one nausea event (grade 3).3 
Food interaction was seen when  darolutamide  formulations (600 mg) were administered after a high fat content breakfast compared to administration in a fasted state.3    AUC and C max  values were approximately two times lower after administration at fast. With high fat meal the C max occurred about 2-3 hours later, indicating a delay in gastric emptying of DAROLUTAMIDE after
NUBEQA ® (  darolutamide) Clinical Data (ARADES & ARAFOR)
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
  
 the high fat meal. The food effect was similar in all groups. The pharmacokinetics observed with the two new tablet formulations were comparable with the pharmacokinetics of the capsule formulation (figure 5).3 
Figure 7: Concentration Time Curves of the Three Formulation with and Without Food3 
Most commonly reported adverse events (>10%) were fatigue/asthenia (13%), nausea (10%), erythema (10%) and urinary tract disorder (10%); all being mild events except one nausea event (grade 3).3 
A Phase 3 trial (ARAMIS) evaluated the safety and efficacy of darolutamide in patients with high risk non-metastatic castrate resistant prostate cancer and can be accessed on the clinicaltrials.gov  https://www.clinicaltrials.gov/ct2/show/NCT02200614?term=ODM+201&rank=4
A Randomized, Double-blind, Placebo Controlled Phase III Study of darolutamide Versus Placebo in Addition to Standard Androgen Deprivation Therapy and Docetaxel in Patients With Metastatic Hormone Sensitive Prostate Cancer has been initiated and listed on the clinicaltrials.gov.  https://www.clinicaltrials.gov/ct2/show/NCT02799602?term=ODM+201&rank=9
   NUBEQA ® (  darolutamide) Clinical Data (ARADES & ARAFOR)
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

REV: 0719
References:
1. NUBEQA®    (darolutamide) tablets for oral use [Prescribing Information]. Whippany, NJ. Bayer
HealthCare Pharmaceuticals, July 2019.
2. Fizazi K, Massard C, Bono P, et al. Activity and safety of ODM-201 in patients with progressive metastatic castration-resistant prostate cancer (ARADES): an open-label phase 1 dose-escalation and randomised phase 2 dose expansion trial.  Lancet Oncol 2  014; 15: 975–85. http://www.ncbi.nlm.nih.gov/pubmed/24974051
3. Massard C, Tammela T, Vjaters, et al. A study of ODM-201 formulations with a safety and tolerability extension phase in patients with metastatic chemotherapy naïve castrate-resistant prostate cancer (CRPC). Presented at 2014 Genitourinary Cancers Symposium, January 2014 in San Francisco CA.
 NUBEQA ® (  darolutamide) Clinical Data (ARADES & ARAFOR)
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.""")

In [104]:
colors = {"DRUG":"#ff9ff0",
         "DISEASE": "turquoise",
         "ADVERSE EVENT": "tomato",
         "CONDITION": "lavender",
         "DOSAGE": "#ffa500",
         "INDICATION": "#ffe6ff",
         "PHARMACOKINETICS": "#b3b3cc",
         "DOSING": "#ffd9b3",
         "EFFICACY": "#d9ffb3",
         "CONTRAINDICATIONS": "#ffa366",
         "SAFETY": "#80ff80",
         "DRUG INTERACTION": "#b3d9ff",
         "FOOD INTERACTION": "#3399ff",
         "BIOAVAILABILITY": "#ff9900",
         "ABSORPTION": "#ccccb3",
         "AUC": "#00ff55",
         "CMAX": "#ff531a",
         "MFS": "#d966ff",
         "OS": "#5c5cd6",
         "TTPP": "#0088cc",
         "PFS": "#ffcccc",
         "QOL": "#cc00cc",
         "OBJECTIVE": "#ffffb3",
         "PLASMA CONCENTRATION": "#ffff1a",
         "DOSAGE FORM": "#f2f2f2",
         "TMAX": "#b3ffff",
         "ENDPOINT": "#ccddff",
         "DISCONTINUATION": "#ccffcc",
         "DEATH": "#ccffe6",
         "ADMINISTRATION": "#ff4da6"}
options = {"ents": ["DRUG", "DISEASE", "ADVERSE EVENT", "CONDITION", "DOSAGE", "INDICATION", "PHARMACOKINETICS", "DOSING", "EFFICACY", "CONTRAINDICATIONS", "SAFETY", "DRUG INTERACTION", "FOOD INTERACTION", "BIOAVAILABILITY", "ABSORPTION", "AUC", "CMAX", "MFS", "OS", "TTPP", "PFS", "QOL", "OBJECTIVE", "PLASMA CONCENTRATION", "DOSAGE FORM", "TMAX", "ENDPOINT", "DISCONTINUATION", "DEATH", "ADMINISTRATION"], "colors": colors}
displacy.serve(doc, style="ent", options=options)

/Users/subashgandyer/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [05/Mar/2020 10:55:43] "GET / HTTP/1.1" 200 104456
127.0.0.1 - - [05/Mar/2020 10:55:44] "GET /favicon.ico HTTP/1.1" 200 104456


Shutting down server on port 5000.


In [92]:
colors = {"DRUG":"#ff9ff0",
         "DISEASE": "turquoise",
         "ADVERSE EVENT": "tomato",
         "CONDITION": "lavender",
         "DOSAGE": "#ffa500",
         "INDICATION": "#ffe6ff",
         "PHARMACOKINETICS": "#b3b3cc",
         "DOSING": "#ffd9b3",
         "EFFICACY": "#d9ffb3",
         "CONTRAINDICATIONS": "#ffa366",
         "SAFETY": "#80ff80",
         "DRUG INTERACTION": "#b3d9ff",
         "FOOD INTERACTION": "#3399ff",
         "BIOAVAILABILITY": "#ff9900",
         "ABSORPTION": "#ccccb3",
         "AUC": "#00ff55",
         "CMAX": "#ff531a",
         "MFS": "#d966ff",
         "OS": "#5c5cd6",
         "TTPP": "#0088cc",
         "PFS": "#ffcccc",
         "QOL": "#cc00cc",
         "OBJECTIVE": "#ffffb3",
         "PLASMA CONCENTRATION": "#ffff1a",
         "DOSAGE FORM": "#f2f2f2",
         "TMAX": "#b3ffff",
         "ENDPOINT": "#ccddff",
         "DISCONTINUATION": "#ccffcc",
         "DEATH": "#ccffe6",
         "ADMINISTRATION": "#ff4da6"}
options = {"ents": ["DRUG", "DISEASE", "ADVERSE EVENT", "CONDITION", "DOSAGE", "INDICATION", "PHARMACOKINETICS", "DOSING", "EFFICACY", "CONTRAINDICATIONS", "SAFETY", "DRUG INTERACTION", "FOOD INTERACTION", "BIOAVAILABILITY", "ABSORPTION", "AUC", "CMAX", "MFS", "OS", "TTPP", "PFS", "QOL", "OBJECTIVE", "PLASMA CONCENTRATION", "DOSAGE FORM", "TMAX", "ENDPOINT", "DISCONTINUATION", "DEATH", "ADMINISTRATION"], "colors": colors}
displacy.render(doc, style="ent", options=options)

### Document 2 - Nubeqa (US-NUBEQA-76044-S.docx-2.pdf)

In [105]:
doc = loaded_nlp("""Phase 1/2 Study of NUBEQA®    (darolutamide) in Patients with Progressive Metastatic Castrate Resistant Prostate Cancer (ARADES)
Summary:
● Nubeqa (darolutamide) is an androgen receptor inhibitor indicated for the treatment of patients with non-metastatic castration-resistant prostate cancer.1 
● Nubeqa is not approved for use in patients with progressive metastatic castrate resistant prostate cancer.
● Phase 1–2 clinical trial was conducted to assess safety and tolerability of darolutamide in men with progressive metastatic castration-resistant prostate cancer.2    Secondary objectives were to determine the pharmacokinetics, and antitumor activity as defined by changes in serum prostate-specific antigen (PSA), by imaging of soft tissue and bone lesions, and by changes in circulating tumor cell counts.
o An open-label, nonrandomized phase 1 dose escalation portion and a phase 2 included randomized dose expansion and long-term follow-up.
o Twenty-four patients were enrolled in the   dose escalation phase. No intra-patient dose escalation was permitted. Patients received 200 mg/day (n=4); 400 mg/day (n=7); 600 mg/day (n=3); 1000 mg/day (n=4); 1400 mg/day (n=3); 1800 mg/day (n=3).
● In the phase 2 part of the study (n=112):
o Thirty-eight patients were assigned to receive 200 mg/day (11 chemotherapy
naïve and CYP17inhibitor (CYP17i) naïve; 9 post chemotherapy and CYP17i
naïve; 18 post CYPi).
o Thirty-seven patients were assigned to receive 400 mg/day (9 [chemotherapy
naïve and CYP17i) naïve]; 9 [post chemotherapy and CYP17i naïve]; 19 [post
CYPi]).
o Thirty-five patients were assigned to receive 1400 mg/day (8 [chemotherapy
naïve and CYP17i) naïve]; 10 [post chemotherapy and CYP17i naïve]; 17 [post
CYPi]).
o Pharmacokinetic analysis darolutamide showed rapid absorption and the median
t max  ranged 3.0–5.1 hours for darolutamide and 1.5–5.0 h for the diastereo isomer keto-darolutamide on day 1. The mean metabolite to parent ratio (area under the curve, AUC) at steady state was 1.6–2.3:1. The exposure of darolutamide (AUC t and C max)  at steady state increased in a linear, dose-related manner up to 1400 mg daily dose.
● Most
(2%) and arthralgia 2 (2%).
common Grade 3 adverse events were fatigue or asthenia 2 (2%), back pain 2
● PSA response (≥ 50% decrease in PSA) by week 12 was observed with all doses and all treatment groups, but significantly fewer were seen in patients previously treated with CYP17 inhibitors than in those who were naïve to both chemotherapy and CYP17 inhibitors.
● RECIST responses or stable disease in soft tissue, and stabilized disease in bone was observed at all dose levels. The highest activity was noted in patients who were chemotherapy-naive and CYP17 inhibitor-naive patients
NUBEQA ®  (darolutamide): ARADES
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
.
 ● Circulating tumor cell measured at baseline and at week 12 (available in 87 of 124 patients) 41 (82%) patients maintained favorable circulating tumor cell counts (< 5 cells per 7·5 mL of blood) at week 12. In the patients with ≥5 cells counts at baseline, 14 (38%) converted to < 5 cells counts, and nine (18%) patients converted from < 5 cells counts to ≥5 cells counts.
● Patients (58%) who were both chemotherapy and CYP17 inhibitor-naïve converted from ≥5 cells counts to < 5 cells counts.
● The median time to PSA progression in phase 1–2 (including all dose levels) was 72.3 weeks (95% CI 24.3–not reached [NR]) for patients who were naive for both chemotherapy and CYP17 inhibitor.
● Median time to radiological progression was not reached for patients who were naive to both chemotherapy and CYP17 inhibitor (95% CI 36.4–NR).
Nubeqa (darolutamide) is an androgen receptor inhibitor indicated for the treatment of patients with non-metastatic castration-resistant prostate cancer.1 
Nubeqa is not approved for use in patients with progressive metastatic castrate resistant prostate cancer.
A Phase 1–2 clinical trial was conducted to assess darolutamide in men with progressive metastatic castration-resistant prostate cancer.2    The primary objectives of the trial were to assess safety and tolerability and to define a maximum tolerated dose. Secondary objectives were to determine the pharmacokinetics, and antitumor activity as defined by changes in serum prostate-specific antigen (PSA), by imaging of soft tissue and bone lesions, and by changes in circulating tumor cell counts.
A proportion of patients in an open-label, multicenter, nonrandomized phase 1 dose escalation study were permitted to continue into a phase 2 part of the study and were randomized in dose expansion and long-term follow-up.2    Patients with adenocarcinoma of the prostate with progressive metastatic disease despite ongoing androgen-deprivation therapy were enrolled.
The phase 1 dose-escalation part of the study was not randomized.2    The patients sequentially entered the phase 1 part of the study. In the phase 2, patients were randomly assigned (ratio 1:1) by blocks of four and two to 200 mg or 400 mg daily doses of darolutamide with previous treatment (chemotherapy-naive and CYP17 inhibitor-naive; post-chemotherapy and CYP17 inhibitor-naive; and post- CYP17 inhibitor) as stratification factors. The third expanded dose level, patients were randomly assigned to 200 mg, 400 mg, or 1400 mg daily doses.
Three to six patients sequentially received 200 mg (100 mg twice daily) which was increased to 400 mg, 600 mg, 1000 mg, 1400 mg, and 1800 mg.2    In the absence of grade 3 or higher treatment-related adverse events within the first 28 days, patients   were permitted to continue darolutamide treatment in the absence of disease progression.2    No   intra-patient dose escalation was permitted.
The phase 2 part of the study included three expanded cohorts (200 mg, 400 mg, and 1400 mg daily dose) each containing about 35 patients.2    A total of 136 patients were enrolled (24 were
NUBEQA ®  (darolutamide): ARADES
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
.

 enrolled in the dose-escalation phase and 112 randomly assigned to receive either 200 mg, 400 mg, or 1400 mg of darolutamide). Of the 24 patients enrolled in dose escalation phase; 200 mg/day (n=4, 1 discontinued protocol violation); 400 mg/day (n=7, 1 discontinued due to AE); 600 mg/day (n=3, 0 discontinued); 1000 mg/day (n=4, 1 discontinued due to AE); 1400 mg/day (n=3, 0 discontinued); 1800 mg/day (n=3, 0 discontinued). Allocation of the 112 patients randomized in the phase 2 part of the study is presented in Figure 1. Two patients randomized did not receive darolutamide.
Figure 1: Trial profile of the phase 2 part of the study2 
 Select baseline characteristic of patients are presented in Table 1.2  Table 1: Select Baseline Characteristics2 
NUBEQA ®  (darolutamide): ARADES
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
.
        Phase1 (n=24)
 Phase 2 Group 1
     Group 2
  Group 3
          200mg (n=38)
      400mg (n=37)
   1400 (n=35)
   Age (years)
      69 (64–74)
 67 (65–73)
      68 (64–75)
   73 (64–78)
   PSA (ng/ml)
      27 (13–96)
 109 (43–352)
      95 (45–255)
   155 (31–382)
 LDH (U/L)
  249 (192–412)
  220 (195–292)
   232 (202–322)
   226 (201–322)
  
                                                                                                                                          Chemotherapy-naive and CYP17 inhibitor-naive
     14 (58
 11 (29%)
     9 (24%)
  8 (23%)
   Post-chemotherapy and CYP17 inhibitor-naïve
      7 (29%)
 9 (24%)
      9 (24%)
   10 (29%)
 Post-CYP17 inhibitor
  3 (13%)
   18 (47%)
    19 (51%)
    17 (49%)
   Total Gleason Score <8
> 8
Missing
    15 (68%) 7 (32%) 2
15 (42%) 21 (58%) 2
    20 (59%) 14 (41%) 3
 15 (46%) 18 (55%) 2
   ECOG PS 0
1
      17 (71%) 7 (29%)
 19 (50%) 19 (50%)
      20 (54%) 17 (46%)
   16 (46%) 19 (54%)
 Circulating tumor cell counts <5 cells/7.5mL of blood
> 5 cells/7.5mL of blood
  13 (54%) 9 (38%)
   18 (47%) 17 (45%)
    18 (49%) 17 (46%)
    16 (46%) 14 (40%)
   Disease localization Bone disease
Bone only
Bone and soft tissue Soft-tissue disease Lymph node Visceral
No radiologic evidence of mets
    15 (63%) 6 (25%) 9 (38%) 17 (71%) 17 (71%) 1 (4%) 1 (4%)
34 (89%) 12 (32%) 22 (58%) 26 (68%) 18 (47%) 10 (26%) 0
    32 (86%) 11 (30%) 21 (57%) 26 (70%) 24 (65%) 8 (22%) 0
 32 (91%) 20 (57%) 12 (34%) 15 (43%) 7 (20%) 8 (23%) 0
 Bone metastases 0
1-4 5-20 >20
  9 (38%) 3 (13%) 7 (29%) 5 (21%)
  6 (16%) 10 (26%) 11 (29%) 11 (29%)
   5 (14%) 13 (35%) 9 (24%) 10 (27%)
   3 (9%) 6 (17%) 8 (23%) 18 (51%)
  PSA=prostate-specific antigen. LDH=lactate dehydrogenase. ECOG=Eastern Cooperative Oncology Group. NA=not assessable, Mets=metastases
The median time on darolutamide therapy was 24-8 (IQR 24.8-25.2) months for the phase 1 part of the study and 11 months (IQR 8.3-14.0) months for phase 2 part of the trial.2    Most common reason for discontinuation was disease progression and 5 (4%) of 124 patients discontinued for adverse events.
Common adverse events occurring in greater than equal to 10% of patients are presented in Table 2 below.2 
Table 2: Most Common Adverse Events Occurring in  >  10% of Patients2 
Fatigue or asthenia Back pain Arthralgia Constipation Decreased appetite Nausea
Peripheral edema
Grade 1
27 (22%) 16 (13%) 15 (12%) 14 (11%) 15 (12%) 12 (10%) 13 (10%)
Grade 2
13 (10%) 13 (10%) 9 (7%) 5 (4%) 1 (<1%) 4 (3%) 2 (2%)
Grade 3
2 (2%) 2 (2%) 2 (2%) 0
0 0 0
NUBEQA ®  (darolutamide): ARADES
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
.

 Pharmacokinetic analysis darolutamide showed rapid absorbed and the median t max  were 3·0–5·1 h for darolutamide and 1·5–5·0 hours for darolutamide on day 1.2    The mean metabolite to parent ratio (area under the curve, AUC) at steady state was 1·6–2·3:1. The exposure of darolutamide (AUC t  and C max)  at steady state increased in a linear, dose-related manner up to 1400 mg daily dose.
Antitumor activity as measured by in PSA from baseline are presented in Figure 2
Figure 2: Percentage in PSA Decline at Week 12 From Baseline2 
 RECIST responses or stable disease in soft tissue, and stabilized disease in bone are shown in table 4.2    No differences in response by dose were noted in soft tissue and bone. This variable was not formally evaluated due to the small number of patients within each dose group and subgroup. The highest activity was noted in patients who were chemotherapy-naive and CYP17 inhibitor-naive patients (table 3).
Table 3: Imaging Results at Week 12 Using RECIST2 
NUBEQA ®  (darolutamide): ARADES
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
.

  Measured baseline and 12-week circulating tumor cell results were available in 87 of 124 patients.2    Circulating conversion rates of circulating tumor cells are presented in table 4 below.
NUBEQA®    (darolutamide): ARADES
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
.

 Table 4: Circulating Tumor Cells at Week 12: Expanded dose Levels2 
        Chemotherapy naïve/CYP17i naïve (n=37*)
 Post Chemotherapy/CYP17i naïve
(n=32*)
    Post CYP 17i (n=55*)
     < at Baseline
    (n=18)
 (n=16)
     (n=25)
     <5 at week 12
    15 (88%)
 10 (91%)
     16 (73%)
     ≥5 at week 12
    2 (12%)
 1 (9%)
     6 (27%)
     NA at week 12
    1
 5
     3
     ≥5 at baseline
    (n=16)
 (n=13)
     (n=24)
     <5 at week 12
    7 (58%)
 7 (58%)
     0
     ≥5 at week 12
    5 (42%)
 ,5 (42%)
     1213 (100%)
 NA at week 12
    4
   1
   11
  NA=not assessable. CTC count is expressed as cells per 7·5 mL of blood. These data include patients with missing baseline CTC. *Total number includes three patients with missing baseline. †   Total number includes six patients with missing baseline.
The median serum testosterone level at baseline was 0·4 nmol/L (IQR 0·4–0·6; n=124) and castrate concentrations were maintained at week 12 (0·4 nmol/L [0·4–0·6]; n=111).2 
The median time to PSA progression in phase 1–2 (including all dose levels) was 72·3 weeks (95% CI 24·3–not reached [NR]) for patients who were naive to both chemotherapy and CYP17 inhibitor; 20·3 weeks (95% CI 16·9–26·1) for those who had received chemotherapy but not a CYP17 inhibitor, and 19·3 weeks (95% CI 14·1–27·1) for those who had received a CYP17 inhibitor (figure 3).2 
NUBEQA®    (darolutamide): ARADES
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
.

 Figure 3: Time to PSA Progression2 
Median time to radiological progression was not reached for patients who were naive to both chemotherapy and CYP17 inhibitor (95% CI 36·4–NR); 50·4 weeks (95% CI 12·1–NR) for those who had received chemotherapy but not a CYP17 inhibitor; and 12·7 weeks (95% CI 12·0–36·3) for those who had received a CYP17 inhibitor (figure 4).2 
Figure 4: Time to Radiologic Progression2 
 NUBEQA®    (darolutamide): ARADES
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
.

  Highest pre-specified daily dose of 1800 mg and a maximum tolerated dose was not reached; no dose-limiting toxic effects were noted and dose escalation was discontinued because of a plasma concentration plateau of darolutamide.
A Phase 3 trial (ARAMIS) evaluated the safety and efficacy of darolutamide in patients with high risk non-metastatic castrate resistant prostate cancer and can be accessed on the clinicaltrials.gov  https://www.clinicaltrials.gov/ct2/show/NCT02200614?term=ODM+201&rank=4
A Randomized, Double-blind, Placebo Controlled Phase III Study of darolutamide Versus Placebo in Addition to Standard Androgen Deprivation Therapy and Docetaxel in Patients With Metastatic Hormone Sensitive Prostate Cancer has been initiated and listed on the clinicaltrials.gov.  https://www.clinicaltrials.gov/ct2/show/NCT02799602?term=ODM+201&rank=9
REV: 0719
References:
1. NUBEQA® (darolutamide) tablets for oral use [Prescribing Information]. Whippany, NJ. Bayer
HealthCare Pharmaceuticals, July 2019.
2. Fizazi K, Massard C, Bono P, et al. Activity and safety of ODM-201 in patients with progressive metastatic castration-resistant prostate cancer (ARADES): an open-label phase 1 dose-escalation and randomised
NUBEQA®    (darolutamide): ARADES
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
.
  
phase 2 dose expansion trial.  Lancet Oncol 2  014; 15: 975–85. http://www.ncbi.nlm.nih.gov/pubmed/24974051
 NUBEQA®    (darolutamide): ARADES
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.""")

In [106]:
colors = {"DRUG":"#ff9ff0",
         "DISEASE": "turquoise",
         "ADVERSE EVENT": "tomato",
         "CONDITION": "lavender",
         "DOSAGE": "#ffa500",
         "INDICATION": "#ffe6ff",
         "PHARMACOKINETICS": "#b3b3cc",
         "DOSING": "#ffd9b3",
         "EFFICACY": "#d9ffb3",
         "CONTRAINDICATIONS": "#ffa366",
         "SAFETY": "#80ff80",
         "DRUG INTERACTION": "#b3d9ff",
         "FOOD INTERACTION": "#3399ff",
         "BIOAVAILABILITY": "#ff9900",
         "ABSORPTION": "#ccccb3",
         "AUC": "#00ff55",
         "CMAX": "#ff531a",
         "MFS": "#d966ff",
         "OS": "#5c5cd6",
         "TTPP": "#0088cc",
         "PFS": "#ffcccc",
         "QOL": "#cc00cc",
         "OBJECTIVE": "#ffffb3",
         "PLASMA CONCENTRATION": "#ffff1a",
         "DOSAGE FORM": "#f2f2f2",
         "TMAX": "#b3ffff",
         "ENDPOINT": "#ccddff",
         "DISCONTINUATION": "#ccffcc",
         "DEATH": "#ccffe6",
         "ADMINISTRATION": "#ff4da6"}
options = {"ents": ["DRUG", "DISEASE", "ADVERSE EVENT", "CONDITION", "DOSAGE", "INDICATION", "PHARMACOKINETICS", "DOSING", "EFFICACY", "CONTRAINDICATIONS", "SAFETY", "DRUG INTERACTION", "FOOD INTERACTION", "BIOAVAILABILITY", "ABSORPTION", "AUC", "CMAX", "MFS", "OS", "TTPP", "PFS", "QOL", "OBJECTIVE", "PLASMA CONCENTRATION", "DOSAGE FORM", "TMAX", "ENDPOINT", "DISCONTINUATION", "DEATH", "ADMINISTRATION"], "colors": colors}
displacy.serve(doc, style="ent", options=options)

/Users/subashgandyer/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [05/Mar/2020 10:57:15] "GET / HTTP/1.1" 200 67049
127.0.0.1 - - [05/Mar/2020 10:57:15] "GET /favicon.ico HTTP/1.1" 200 67049


Shutting down server on port 5000.


In [93]:
colors = {"DRUG":"#ff9ff0",
         "DISEASE": "turquoise",
         "ADVERSE EVENT": "tomato",
         "CONDITION": "lavender",
         "DOSAGE": "#ffa500",
         "INDICATION": "#ffe6ff",
         "PHARMACOKINETICS": "#b3b3cc",
         "DOSING": "#ffd9b3",
         "EFFICACY": "#d9ffb3",
         "CONTRAINDICATIONS": "#ffa366",
         "SAFETY": "#80ff80",
         "DRUG INTERACTION": "#b3d9ff",
         "FOOD INTERACTION": "#3399ff",
         "BIOAVAILABILITY": "#ff9900",
         "ABSORPTION": "#ccccb3",
         "AUC": "#00ff55",
         "CMAX": "#ff531a",
         "MFS": "#d966ff",
         "OS": "#5c5cd6",
         "TTPP": "#0088cc",
         "PFS": "#ffcccc",
         "QOL": "#cc00cc",
         "OBJECTIVE": "#ffffb3",
         "PLASMA CONCENTRATION": "#ffff1a",
         "DOSAGE FORM": "#f2f2f2",
         "TMAX": "#b3ffff",
         "ENDPOINT": "#ccddff",
         "DISCONTINUATION": "#ccffcc",
         "DEATH": "#ccffe6",
         "ADMINISTRATION": "#ff4da6"}
options = {"ents": ["DRUG", "DISEASE", "ADVERSE EVENT", "CONDITION", "DOSAGE", "INDICATION", "PHARMACOKINETICS", "DOSING", "EFFICACY", "CONTRAINDICATIONS", "SAFETY", "DRUG INTERACTION", "FOOD INTERACTION", "BIOAVAILABILITY", "ABSORPTION", "AUC", "CMAX", "MFS", "OS", "TTPP", "PFS", "QOL", "OBJECTIVE", "PLASMA CONCENTRATION", "DOSAGE FORM", "TMAX", "ENDPOINT", "DISCONTINUATION", "DEATH", "ADMINISTRATION"], "colors": colors}
displacy.render(doc, style="ent", options=options)

### Document 3 - Nubeqa(US-NUBEQA-76047-S.docx.pdf)

In [110]:
doc = loaded_nlp("""NUBEQA®   (darolutamide) and Central Nervous System-related Adverse Events
Nubeqa (darolutamide) is an androgen receptor inhibitor indicated for the treatment of patients with non-metastatic castration-resistant prostate cancer.1 
Background
Darolutamide is a second-generation Androgen Receptor (AR) inhibitor.2    Second-generation AR inhibitors share antagonist activity on GABA-A, a ligand-gated chloride ion channel. Inhibition of this channel has been suggested to induce seizures.3    This mechanism is, however, dependent on local concentrations in the central nervous system (CNS). Prior to darolutamide, second-generation AR inhibitors evaluated in placebo-controlled randomized trials in patients with high-risk non-metastatic Castration-Resistant Prostate Cancer (nmCRPC) reported significant CNS-related events, despite exclusion of patients with a history of CNS-related events.
According to the Center for Disease Control in prevention, approximately 1.2 % of the general United States population were reported as having epilepsy. Although the prevalence of epilepsy appears low, the Institute for Clinical and Economic Review (ICER) has identified CNS-related events such as falls, fractures, and seizures to be treatment emergent adverse events (TEAEs) of interest for anti-androgen therapies.4 
This letter will discuss CNS-related events reported in trials with darolutamide including the most recent Phase III, randomized, double-blind, placebo-controlled multinational study, evaluating the efficacy and safety of darolutamide in men being treated with androgen deprivation therapy (ADT) for non-metastatic castration-resistant prostate cancer (nmCRPC) and have a high risk of developing metastases -- ARAMIS.  2,3,5-9
Summary
● Nubeqa (darolutamide) is an androgen receptor inhibitor indicated for the treatment of patients with non-metastatic castration-resistant prostate cancer.1 
● ARAMIS, a Phase III, randomized, double-blind, placebo-controlled multinational study, evaluated the efficacy and safety of darolutamide in men being treated with androgen deprivation
therapy (ADT) for non-metastatic castration-resistant prostate cancer (nmCRPC) and have a high risk of developing metastases. The primary endpoint of the study was metastasis-free survival (MFS). MFS was defined as the time from randomization to evidence of metastasis (new or progressive regional pathological lymph nodes were not defined as metastasis) or death from any cause, whichever occurred first.
o Darolutamide demonstrated a statistically significant improvement in MFS of 40.4 months [95% CI 34.33 – NR] vs 18.4 months [95% CI 15.51 – 22.34] in the placebo arm and reduced the risk of metastasis or death by 59% versus placebo [HR .41 (95% CI 0.34 – 0.50);  P  <0.001]. The most common adverse reactions (≥2%) reported were fatigue, pain in extremity, and rash.  5 
NUBEQA ®  (darolutamide): CNS-related AEs
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
 ● In addition to evaluation of metastasis-free survival (MFS), safety was evaluated, including CNS-related events.    Patients with a history or risk of a seizure were included.6 
o The most frequent CNS-related TEAEs (all grades) occurring in ≥5% of patients included fatigue (including asthenia) (12.1% [115/954] vs 8.7% [48/554] in the darolutamide vs placebo arms, respectively).7 
o Most of the adverse events reported were Grade 1 or 2 (54.6% with darolutamide and 54.2% with placebo).
o Grade 3 fatigue including asthenia was reported in 0.6% of patients in the darolutamide arm vs 1.1% in the placebo arm.7  7
o No Grade 4 or Grade 5 CNS-related adverse event occurred in either arm.  ● Phase I and II (ARADES study) enrolled 134 patients with progressive metastatic
castration-resistant prostate cancer (mCRPC). Patients with a history, or risk, of a seizure were included.2 
o Data from the ARADES safety database were reviewed for potential seizure or seizure-related adverse events2 
▪ The following adverse event (AE) terms were used identify patients with AEs potentially related to seizures: seizure, syncope, presyncope, loss of consciousness, depressed level of consciousness, encephalopathy, and transient ischemic attack, hallucinations, vasovagal syndrome, nonpathologic fractures, urinary or fecal incontinence, tonic-clonic activity, abnormal electroencephalograms (EEGs).2 
▪ Medical histories and use of concomitant anti-seizure medications were reviewed, as well as concomitant medications that are indicated for treatment of any seizure-related disorders. Medications that could lower the seizure threshold were also identified, along with any seizure or seizure-like condition.2 
o Twenty one percent of the patients had concomitant medications indicated for epilepsy or other seizure-related disorders.2    Twelve patients had  > 1 of the above listed AE. None of the patients experienced seizures or related disorders while receiving darolutamide; one seizure event was reported to occur 27 days after stopping treatment. There were 5 fall events, 1 syncope, 1 presyncope, and 4 urinary incontinence cases – all interpreted by the investigators as not related to darolutamide treatment and not leading to treatment discontinuation. Careful evaluation of these cases suggested that these events were unlikely to be related to seizure.2 
● ARAFOR, an open-label randomized phase I trial, evaluated the bioavailability and effect of food on the bioavailability of darolutamide. The study compared relative bioavailability of 600 mg tablets (tablet A and tablet B) and capsule (reference) formulations in patients with chemotherapy-naïve mCRPC. In addition, the study evaluated short term safety and tolerability and in an extension component long-term safety and tolerability.8 
o Thirty men with metastatic chemotherapy-naïve CRPC were enrolled; the median age of the patients was 68 years.
NUBEQA ®  (darolutamide): CNS-related AEs
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
 
 o A total of 22 of 30 patients (73%) reported AEs; most were grade 1–2 (114 of 125 [91%]).
▪ The most common reported CNS-related AE was fatigue (all grade 1; n = 4/30 [13%]).
▪ No dose reductions were necessary.
▪ No seizures were reported.
● In preclinical mouse and rat models, darolutamide showed negligible brain penetration at
pharmacological doses.2    Brain/plasma ratio of darolutamide’s main active metabolite, keto-darolutamide, was also negligible after darolutamide administration (Table 5). Quantitative whole body autoradiography (QWBA) in rats confirmed the minimal penetration of Darolutamide into the brain.3    In rodent studies, brain exposures to darolutamide in terms of AUC (0-24) were very low with 4.5% of plasma exposure after single dose and 2.1 - 3.9% after repeated doses. Darolutamide has negligible brain penetration in nonclinical models.2 
Clinical Data
Darolutamide has been shown to have negligible brain penetration at pharmacological doses in preclinicalmodels.2   Consistentwiththispre-clinicalobservation,minimalreportsofCNSevents were observed in clinical trials, including ARAMIS,6    ARADES2    and ARAFOR.8    See brief summaries of these studies below.
Non-metastatic Castration-resistant Prostate Cancer (nmCRPC)
ARAMIS, a Phase III, randomized, double-blind, placebo-controlled multinational study, evaluated the efficacy and safety of darolutamide in men being treated with androgen deprivation therapy (ADT) for non-metastatic castration-resistant prostate cancer (nmCRPC) and have a high risk of developing metastases.7    Patients (N = 1509) were randomized 2:1 to receive darolutamide 600 mg orally twice daily plus ADT versus matching placebo. In addition to evaluation of metastasis-free survival as the primary efficacy variable, safety was evaluated including CNS-related events.    Patients with a history or risk of a seizure were included.8 
Overall, 12 patients in the darolutamide arm had a medical history of seizure, including 2 patients (0.2%) with reported episodes prior to entering the study and 10 patients (1.0%) with ongoing epilepsy. In the placebo arm, 1 patient (0.2%) reported having post-traumatic epilepsy, which was ongoing at study entry.6    None of the patients with a medical history of seizure had a seizure during the trial. The most frequent CNS-related TEAEs (all grades) occurring in ≥ 5% of patents included fatigue (including asthenia) (12.1% [115/954] vs 8.7% [48/554] in the darolutamide vs placebo arms, respectively). See Table 1 below.7 
Most adverse events reported were Grade 1 or 2 (54.6% with darolutamide and 54.2% with placebo). Grade 3 fatigue including asthenia (0.6% vs 1.1%) were reported in the darolutamide vs placebo arm, respectively. No Grade 4 or Grade 5 CNS-related adverse event occurred in either arm. See Table 1.7 
NUBEQA ®  (darolutamide): CNS-related AEs
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 For the incidence of darolutamide CNS-related TEAEs of particular interest in the nonmetastatic castration resist prostate cancer space (nmCRPC) see Table 2 .7 
Table 1
Table 2
    ARAMIS: Total of CNS- related TEAEs with an Incidence of  ≥ 5%7 
     Preferred term, n (%)
   Darolutamide N = 954
  Placebo N = 554
     Fatigue
    115 (12.1)
   48 (8.7)
 Hot flush
    50 (5.2)
    23 (4.2)
        ARAMIS: CNS-related TEAEs of Interest7 
    Darolutamide N=954
Placebo N=554
                        AE
Any Grade, n (%)
Grade 3, %
Grade 4 %
Grade 5, %
Any Grade, n (%)
Grade 3, %
Grade 4 %
Grade 5, %
                                Fatigue, including asthenia
151 (15.8)
0.6
0
0
63 (11.4)
1.1
0
0
                    Seizure
2 (0.2)
0
1 (0.2)
0
                      Cognitive disorder
4 (0.4)
0
0
0
1 (0.2)
0
0
0
                                Memory impairment
Disturbance in attention
5 (0.5)
0
0
0
7 (1.3)
0
0
0
                                0
0
0
0
1 (0.2)
0
0
0
                                Amnesia
1 (0.1)
0
0
0
3 (0.5)
0
0
0
                    Hot flush
50 (5.2)
0
-
-
23 (4.6)
0
-
-
            Metastatic Castration-Resistant Prostate Cancer (mCRPC)
The ARADES trial, an open-label, multicenter trial with a non-randomized Phase I dose escalation portion and a Phase II randomized dose expansion phase, enrolled patients with progressive, metastatic castration-resistant prostate cancer (mCRPC).2    Patients with a history of seizures or were at risk of seizures were not excluded. Concomitant medications that may have lowered the seizure threshold were not an exclusion criteria for the patients in the trials. To assess whether treatment with Darolutamide is possibly associated with seizures, the ARADES safety database was reviewed for potential seizure or seizure-related AEs.
The safety data were reviewed using the following AE terminology in order to identify patients with AE potentially related to seizures were: seizure, syncope, presyncope, loss of consciousness, depressed level of consciousness, encephalopathy, transient ischemic attack, hallucinations, vasovagal syndrome, urinary or fecal incontinence, tonic-clonic activity and abnormal EEGs.2 
NUBEQA ®  (darolutamide): CNS-related AEs
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 Patients’ medical histories and use of concomitant anti-seizure medications, as well as concomitant medications that were indicated for the treatment of any seizure-related disorders were reviewed. Medications that lower the seizure threshold or induce seizure-like conditions were also reviewed.2 
Patients (N = 134) in the ARADES trial had progressive, mCRPC with serum testosterone levels <30 ng/mL. They were either chemotherapy-naïve or had ≤2 prior chemotherapy regimens. Baseline characteristics are presented in Table 3.2 
Table 3: Baseline Characteristics.2 
        Total N=134
   CYP17i-naïve, chemo-naïve, N
     42 (31.3)
   CYP17i-naïve, post-chemo
     35 (26.1)
   Post-CYP17i
     57 (42.5)
   Age, median years (range)
     60 (53-89)
   PSA ng/mL, median (range)
     99 (3-5000)
   Time from diagnosis to study treatment, median months (range)
    69 (10-231)
 Disease localization, N (%): Bone disease only
Soft Tissue only
Soft and Bone Disease
   49 (36.6) 20 (14.9) 64 (47.8)
   Seizure (and like) event history %
    11 (8.2)
 Time on Study Drug (days) Mean (SD)
Median (min, max)
  184 (131.0) 168 (1,684)
In the ARADES trial, there were 38 medications with indications to treat epilepsy or other seizure-related disorders (11 insomnia, 8 anxiety, 10 pain, 3 epilepsy, 2 depression, 1 polyneuropathy, 1 alcohol abuse, 1 bowel incontinence, and 1 restless leg syndrome).2 
Sixteen patients reported at least one of the prespecified seizure-related AEs mentioned above (Table 4).2    There were 1 syncope, 1 presyncope, and 5 urinary incontinence, 2 collapses, 2 fecal incontinence cases that were all considered as not drug-related by the investigator.2    Only one report of urinary incontinence that occurred 256 days after the start of treatment with 700 mg twice daily was considered drug-related. Other than that, none of these AEs was considered a seizure-like event and none led to study drug discontinuation.2 
None of the patients experienced seizures or related disorders while receiving darolutamide.2    A seizure event was reported after the patient had his end of trial visit, 27 days after stopping the darolutamide. This patient had collapses in his medical history and one during the trial (Table 4), along with anemia, which was noted by the investigators to be a confounder of this event.
NUBEQA ®  (darolutamide): CNS-related AEs
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 Table 4. Central Nervous System-Related Adverse Events During Study.2 
  Phase
    Dose
   Event
   Days in Study
   Relevant Medical History
     I
 500 mg
Urinary incontinence
    0
 Stress urinary incontinence due to prostatectomy
     I
    300 mg
 Presyncope
      7
   Presyncope 15 yrs ago
     II
    200 mg
 Fecal incontinence
      47
   Fecal incontinence
     II
    700 mg
 Urinary incontinence
      256
        II
    200 mg
 Urinary incontinence
      255
   Oxazepam
 II
     700 mg
    Urinary incontinence
    229
         II
 200 mg
Collapse
    86
 Collapses in medical history (one during trial) Anemia
     II
    100 mg
 Collapse
      220
    II
     100 mg
    Fecal and urinary incontinence
    65
       II
  100 mg
 Syncope
  218
 Temazepam for insomnia Lorazepam, alcohol abuse
 The ARAFOR trial, an open-label, randomized, multicenter phase I trial, evaluated the bioavailability and effect of food on the bioavailability of darolutamide. The study compared relative bioavailability of 600 mg tablets (tablet A and tablet B) and capsule (reference) formulations in patients with chemotherapy-naïve mCRPC. The study also assessed the effect of food on the bioavailability of darolutamide tablet products tablet A (TabA) and tablet B (TabB). In addition, the study evaluated short term safety and tolerability and in an extension component long-term safety and tolerability. Thirty men with metastatic chemotherapy-naïve CRPC were enrolled; the median age of the patients was 68 years.8 
The pharmacokinetic (PK) component consisted of two concurrent groups, which evaluated TabA and (TabB), and capsule. Patients were randomized to one of 3 treatment sequences in each group. There were three treatment periods for each patient during the PK Component which included administration of two darolutamide 300 mg test tablet (TabA or TabB) in both fasted and fed conditions, or administration of darolutamide 100 mg capsule at 600 mg dose in fed state. There was at least 7 days wash-out period between the three study treatment administrations. The total daily dose was 600 mg/day.8 
All patients completed the PK component and entered the extension component at the 4-week visit. During the extension, patients were treated with a darolutamide 600 mg capsule taken with food twice daily. Treatment continued until disease progression or an intolerable AE.8 
NUBEQA®    (darolutamide): CNS-related AEs
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

All 30 patients completed the first 12 weeks in the extension. A total of 22 of 30 patients (73%) reported AEs; most were grade 1–2 (114 of 125 [91%]). The most common CNS-related AE was fatigue (all grade 1) in 4 patients (13%). Headache was reported as an AE in 2 patients (7%). No dose reductions were made and no seizures were reported.8 
Pre-Clinical Data
In preclinical, rat and mouse, models, darolutamide showed negligible brain penetration at pharmacological doses.2    Brain/plasma ratio of the main active metabolite, keto-darolutamide, was also negligible after darolutamide administration (Table 5). Quantitative whole body autoradiography (QWBA) in rats confirmed the minimal penetration of darolutamide into the brain. In rodent studies, brain exposures to darolutamide in terms of AUC (0-24) were very low with 4.5% of plasma exposure after single dose and 2.1 - 3.9% after repeated doses. Darolutamide has negligible brain penetration in nonclinical models.  2
Table 5: Brain Penetration of Darolutamide.2 
REV: 07/2019 References:
1. NUBEQA®    (darolutamide) tablets for oral use [Prescribing Information]. Whippany, NJ. Bayer HealthCare Pharmaceuticals, July 2019.
2. Shore ND, Bono P, Massard C, et al. J Clin Oncol 32, 2014 (suppl 4; abstr 275) Presented at the 2014 Genitourinary Cancers Symposia. San Francisco, CA
3. Foster, WR, Car, BD, Shi, H, et al. Drug safety is a barrier to the discovery and development of new androgen receptor antagonists. Prostate, 2011; 71: 480–488.  https://www.ncbi.nlm.nih.gov/pubmed/20878947
4. Institute for Clinical and Economic Review (ICER), 2018
5. Clegg NJ, Wongvipat J, Joseph J, et al. ARN-509: a novel anti-androgen for prostate cancer treatment.  Cancer
Res . 2012;72(6):1494-1503.  https://www.ncbi.nlm.nih.gov/pubmed/22266222
6. Data on file, Bayer Pharmaceutical, Whippany, NJ
7. Fizazi K et al. Darolutamide in nonmetastatic prostate cancer Oral presentation at: [ASCO GU]; February 14 – 16, 2019; San Francisco CA.
8. Institute for Clinical and Economic Review (ICER), 2018
9. Massard C, Penttinen HM, Vjaters E, et al. Pharmacokinetics, antitumor activity, and safety of ODM-201 in
patients with chemotherapy-naive metastatic castration-resistant prostate cancer: an open-label phase 1 study. Eur Urol. 2016 May;69(5):834-40.  https://www.ncbi.nlm.nih.gov/pubmed/26463318
NUBEQA®    (darolutamide): CNS-related AEs
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.""")

In [111]:
displacy.render(doc, style="ent", options=options)

In [112]:
displacy.serve(doc, style="ent", options=options)

/Users/subashgandyer/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [05/Mar/2020 10:58:53] "GET / HTTP/1.1" 200 82406
127.0.0.1 - - [05/Mar/2020 10:58:54] "GET /favicon.ico HTTP/1.1" 200 82406


Shutting down server on port 5000.


### Document 4 - Nubeqa (US-NUBEQA-76049-S.docx-2.pdf)

In [113]:
doc = loaded_nlp("""NUBEQA®    (darolutamide) Safety Data
Nubeqa (darolutamide) is an androgen receptor inhibitor indicated for the treatment of
patients with non-metastatic castration-resistant prostate cancer.1 
Background
Darolutamide is a second-generation Androgen Receptor (AR) inhibitor given concurrently with androgen deprivation therapy (ADT) for the treatment of non-metastatic castration-resistant prostate cancer (nmCRPC).1 
There is a need for agents that not only provide significant clinical benefit but also have a tolerable safety profile with minimal detriment to patient quality of life.2    Darolutamide is structurally distinct from other second-generation AR inhibitors. It is a more polar molecule with a flexible structure. Increased flexibility, higher polarity, and increased hydrogen bond-forming potential are associated with low blood brain barrier penetration in preclinical models.7  .
This letter will discuss adverse events reported in the ARAMIS, ARADES and ARAFOR trials.3  -6
Summary
● Nubeqa (darolutamide) is an androgen receptor inhibitor indicated for the treatment of patients with non-metastatic castration-resistant prostate cancer.1 
● ARAMIS, a Phase III, randomized, double-blind, placebo-controlled multinational study, evaluated the efficacy and safety of darolutamide in men being treated with androgen deprivation therapy (ADT) for non-metastatic castration-resistant prostate cancer (nmCRPC) and have a high risk of developing metastases. The primary endpoint of the study was metastasis-free survival (MFS). MFS was defined as the time from randomization to evidence of metastasis (new or progressive regional pathological lymph nodes were not defined as metastasis) or death from any cause, whichever occurred first.3  -4
o Darolutamide demonstrated a statistically significant improvement in MFS of 40.4 months [95% CI 34.33 – NR] vs 18.4 months [95% CI 15.51 – 22.34] in the placebo arm and reduced the risk of metastasis or death by 59% versus placebo [HR .41 (95% CI 0.34 – 0.50);  P  <0.001].  3
● In addition to evaluation of Metastasis-free Survival as the primary efficacy variable, safety was evaluated.
o After a median exposure of 14.8 months in the darolutamide arm versus 11.04 months in the placebo arm, approximately 80% of patients experienced an adverse event during the trial (N = 1220/1508; [darolutamide arm, n = 794; placebo arm, n = 426]). The majority of adverse events reported in both arms were Grade 1 or 2 [54.4% (821/1508)].3    The incidence of common adverse events were reported in ≤10% of patients in both the darolutamide and placebo arms, with only fatigue (including asthenia) reported at an incidence of >10% in both arms.3 
Exposure-adjusted Adverse Events
Darolutamide Safety Data
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
 ▪ After adjustment for duration of treatment or observation period (median exposure of 14.8 months in the darolutamide arm versus 11.04 months in the placebo arm), differences in the incidence of adverse events of interest between both arms decreased or were no longer evident.3 
o The most common Grade 3 or 4 events were hypertension (3.1% of patients in the darolutamide plus ADT arm versus 2.2% of patients in the placebo plus ADT arm.) and urinary retention (1.6% of patients in the darolutamide plus ADT arm versus 2.0% of patients in the placebo plus ADT arm.).3 
o Treatment discontinuations occurred in 35.5% (339/954) of patients in the darolutamide arm vs. 63.9% (354/554) of patients in the placebo arm.3 
● ARADES, an open-label, multicenter trial with a non-randomized Phase I dose escalation portion and a Phase II randomized dose expansion phase, enrolled patients (N = 134) with progressive, metastatic castration-resistant prostate cancer (mCRPC).5    Patients with a history of seizures or were at risk of seizures were included.5 
o In the placebo arm, treatment discontinuations due to adverse events occurred in 8.5% (47/554) of patients; treatment discontinuations due to metastasis occurred in 23.3% (129/554) of patients.5 
o The most common adverse events (all grades) were fatigue including asthenia (31%, arthralgia (16%) and pain (15%). Most adverse events reported were Grade 1 or 2.5 
o Grade 3 and 4 adverse events reported in 22% (n = 27) and <2% (n = 2) patients, respectively. Darolutamide was tolerated up to the highest dose studied (1800 mg/day) and the maximum tolerated dose was not reached. None of the reported events were deemed by the investigator to be drug-related. No seizures were reported.5 
o Adverse events considered drug-related were reported in 35% (n = 44) patients and included fatigue including asthenia in 12% (n = 15) of patients, hot flushes in 5% (n = 6) of patients, decreased appetite in 4% (n = 5) of patients and diarrhea in 2% (n = 3) of patients.5 
▪ One grade 3 event (fatigue including asthenia) was deemed by the investigator to be possibly drug-related. No grade 4 events were deemed to be drug-related.5 
● ARAFOR, a two-part, multicenter, international, open-label, Phase I study, evaluated the pharmacokinetic (PK), safety and tolerability of darolutamide (ODM-201) in patients (N = 30) with progressive metastatic castration resistant prostate cancer (mCRPC). Patients with a history of seizures or were at risk of seizures were included.6 
o After a median reporting period of 15.3 months (95% CI 9.7 months – NR), 73% (22/30) of patients reported adverse events.6 
o The majority of the AE’s reported were Grade 1 or 2. The most common AEs were reported included fatigue (13%, n = 4) [All grade 1] and nausea (13%, n = 4) [All grade 1 – 3].6 
o Twenty percent (20%) of AEs reported were considered treatment-related.
Darolutamide Safety Data
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 ▪ Treatment-related AEs were fatigue, decreased appetite, headache, abdominal pain, solar dermatitis, tinnitus, and dysgeusia. All treatment-related AEs were Grade 1.6 
o No seizures were reported.6  6 o No dose reductions were necessary. 
Clinical Data
Non-metastatic Castration-Resistant Prostate Cancer (nmCRPC) ARAMIS Trial3  -4
Study Design
ARAMIS, a Phase III, multinational, randomized, double blind, placebo-controlled study, evaluated the efficacy and safety of darolutamide in men undergoing treatment with androgen deprivation therapy (ADT*   ) for non-metastatic castration-resistant prostate cancer (nmCRPC) with a high risk of developing metastases. High risk was defined as PSADT of ≤ 10 months and PSA ≥ 2ng/ml. Patients included in the study had an ECOG performance status of 0-1 and adequate hematologic, renal, and liver function. Additionally, patients with a history of seizures or at risk for seizures were included.3 
Patients (N=1509) were randomized 2:1 to receive darolutamide 600 mg tablets orally twice daily with food plus ADT versus matching placebo plus ADT. Key baseline characteristics of the majority of the patient population in the ARAMIS study were: a median age of 74; Gleason score of ≥7; absence of regional pathological lymph nodes; ≥2 prior hormonal therapy agents with a PSADT ≤6 months, and no baseline osteoclast targeted therapy. Patients on the ARAMIS study were stratified according to PSADT (PSADT;  <  6 months vs. > 6 months) and use of osteoclast-targeted therapy (yes vs. no). All assessments were obtained at baseline and every 16 weeks.3 
The primary endpoint of the study was metastasis free-survival (MFS) [defined as time between randomization and evidence of metastasis or death from any cause, whichever occurs first. Secondary endpoints included OS, Time to Pain Progression, Time to Initiation of First Cytotoxic Chemotherapy and Time to First Symptomatic Skeletal Event (SSE). Exploratory endpoints included PFS and Time to PSA Progression.3 
Safety Results3 
After a median exposure of 14.8 months in the darolutamide arm versus 11.04 months in the placebo arm, approximately 80% of patients experienced an adverse event during the trial
(N = 1220/1508; [darolutamide arm, n = 794; placebo arm, n = 426]). The incidence of adverse events were generally similar between treatment arms and the majority were Grade 1 or 2 [54.4% (821/5108)].3    The incidence of common events was low with only fatigue reported at an incidence of >10%. Grades 3 or 4 adverse events occurred in 22.8% (344/1508) of the total safety population (24.7% of patients in the darolutamide arm versus 19.5% of patients in the placebo arm). The most frequent Grade 3 or 4 adverse events included hypertension (3.1% of patients in the darolutamide plus ADT arm versus 2.2% of patients in the placebo plus ADT
Darolutamide Safety Data
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
 
 arm.) and urinary retention (1.6% of patients in the darolutamide plus ADT arm versus 2.0% of patients in the placebo plus ADT arm.).3 
Table 1 shows the Treatment Emergent Adverse Events (TEAEs) for the total safety population (N = 1508). TEAEs (all grades) occurring in ≥5% of patients reported in Table 6 below, in the darolutamide versus placebo arms, respectively, included fatigue (including asthenia) (12.1% [115/954, vs 8.7% [48/554]). Other TEAEs occurring in ≥5% in the darolutamide arm included back pain (8.8%), arthralgia (8.1%), diarrhea (6.9%), hypertension (7.3%), constipation (TEAEs occurring in 6.3%), pain in extremity (5.8%), anemia (5.6%) and hot flush (5.2%).3  Approximately 14% of TEAEs led to a dose modification in the darolutamide arm versus 9.4% of TEAEs in the placebo arm.
After a median follow-up of 17.9 months, treatment discontinuations were similar between both arms.4    Discontinuations due to adverse events occurred in 8.9% of patients in the darolutamide arm vs 8.7% of patients in the placebo arm. Discontinuations due to metastasis occurred in 11.7% vs 23.3% of patients; and discontinuations due to investigator judgment occurred in 5.7% vs 16.4% of patients, in the darolutamide vs placebo arms, respectively. Discontinuations due to protocol deviation and to other reasons unspecified occurred in 1.4% vs 1.3% and 0.6% vs 0.4% in the darolutamide vs placebo arm, respectively. Additionally, 7.1% vs 14.1% of patients in the darolutamide vs placebo arm respectively decided not to continue trial participation. It should be noted that one person in the darolutamide arm was randomized but did not receive treatment.4 
A total of 55 deaths occurred during the trial with 3.9% of deaths occurring in the darolutamide arm versus 3.2% occurring in the placebo arm.3    See Table 1 below.
The Institute for Clinical and Economic Review (ICER) has identified events such as falls, fractures, cardiovascular events and deaths due to AE’s, etc. to be TEAE’s of interest for anti-androgen therapies.2    For the incidence of TEAE’s of interest reported for darolutamide, see Tables 2 - 4.3 
    Table 1. ARAMIS: Summary of TEAEs3 
      Darolutamide N = 954
     Placebo N = 554
  Total N = 1508
     Exposure, median (months)
 14.8
      11.04
        Any AE, n (%)
 794 (83.2)
      426 (76.9)
   1220 (80.9)
     Worst grade
               Grade 1 or 2
 521 (54.6)
      300 (54.2)
   821 (54.4)
 Grade 3 or 4
   236 (24.7)
    108 (19.5)
    344 (22.8)
     Grade 5
37 (3.9)
    18 (3.2)
 55 (3.6)
     Serious
 237 (24.8)
      111 (20.0)
   348 (23.1)
 Leading to dose modification
   135 (14.2)
    52 (9.4)
    187 (12.4)
   Leading to permanent discontinuation
85 (8.9)
  48 (8.7)
 133 (8.8)
           AE, adverse event, TEAE, treatment emergent adverse event
Table 2. ARAMIS: Total of TEAEs with an Incidence of  ≥ 5%3 
Darolutamide Safety Data
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
     
                                                                                                                    Preferred term, n (%)
   Darolutamide N = 954
  Placebo N = 554
     Fatigue
    115 (12.1)
   48 (8.7)
     Back pain
    84 (8.8)
   50 (9.0)
     Arthralgia
    77 (8.1)
   51 (9.2)
     Diarrhea
    66 (6.9)
   31 (5.6)
     Hypertension
    70 (7.3)
   33 (6.0)
     Constipation
    60 (6.3)
   34 (6.1)
     Pain in extremity
    55 (5.8)
   18 (3.2)
     Anemia
    53 (5.6)
   25 (4.2)
     Hot flush
    50 (5.2)
   23 (4.2)
     Nausea
    48 (5.0)
   32 (5.8)
     Bone Fracture
    40 (4.2)
   21 (3.8)
     Fall
    40 (4.2)
   26 (4.7)
     Rash
    24 (2.6)
   4 (0.7)
 Weight decreased
    34 (3.6)
    12 (2.2)
                   Table 3. ARAMIS: TEAEs of Interest3 
    Darolutamide N=954
Placebo N=554
                        AE
Any Grade, n (%)
Grade 3, %
Grade 4 %
Grade 5, %
Any Grade, n (%)
Grade 3, %
Grade 4 %
Grade 5, %
                                Bone Fracture
40 (4.2)
0.9
0
0
21 (3.8)
0.9
0
0
                    Fall
40 (4.2)
0.8
0
0
26 (4.7)
0.7
0
0
                      Fatigue, including asthenia
151 (15.8)
0.6
0
0
63 (11.4)
1.1
0
0
                    Seizure
2 (0.2)
0
1 (0.2)
0
                      Cognitive disorder
4 (0.4)
0
0
0
1 (0.2)
0
0
0
                                Memory impairment
5 (0.5)
0
0
0
7 (1.3)
0
0
0
                                Disturbance in attention
0
0
0
0
1 (0.2)
0
0
0
                                Amnesia
1 (0.1)
0
0
0
3 (0.5)
0
0
0
                    Rash
24 (2.6)
0.1
0
0
4 (0.7)
0
0
0
                      Weight decreased
34 (3.6)
0
0
0
12 (2.2)
0
0
0
                    Hot flush
50 (5.2)
0
-
-
23 (4.6)
0
-
-
                 Increased bilirubin
18 (1.9)
0.1
0
0
0
0
0
0
                 Table 4. AE
Hypertension
ARAMIS: TEAEs of Interest3    continued Darolutamide
N=954 Any Grade 3, Grade 4 Grade 5,
Placebo N=554
n (%) n (%)
70 (7.3) 3.4 0 0 33 (6.0) 2.3 0 0
Any
Grade, % % % Grade, % % %
Darolutamide Safety Data
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
Grade 3,
Grade 4 Grade 5,

    Cerebral and intracranial hemorrhage
  2 (0.2)
  0
     0
   0
 2 (0.4)
  0.2
     0
  0.2
 Cerebral infarction and stroke
  4 (0.4)
  0.3
  0
     0
    4 (0.7)
    0.4
  0
    0.2
   Cerebral ischemic infarction and stroke
 5 (0.5)
 0.1
    0.2
 0.1
2 (0.4)
 0.2
    0
 0.2
   Acute myocardial infarction
   5 (0.5)
   0.2
      0.2
    0.1
 1 (0.2)
   0
      0
   0
 Cardiac failure
  18 (1.9)
  0.1
  0.1
     0.3
    5 (0.9)
    0
  0
    0.5
   Coronary artery disorders
 31 (3.2)
 1.3
 0.4
   0.3
  14 (2.5)
  0.4
 0
  0.2
  Ischemic coronary artery disorders
 24 (2.5
 0.7
 0.4
  0.2
 11 (2.0)
  0.4
 0
 0.2
       Exposure-adjusted Adverse Events
After adjustment for duration of treatment or observation period (After a median exposure of 14.8 months in the darolutamide arm versus 11.04 months in the placebo arm), differences in the incidence of adverse events of interest between both arms decreased or no longer evident.3  See table 5 below.
    Table 5 Exposure-adjusted Adverse events3  ( patients per 100 years of exposure )
     Adverse events of interests
   Darolutamide Exposure-adjusted incidence (N = 954)
  Placebo Exposure-adjusted incidence (N = 554)
     Fatigue or asthenic conditions
    11.3
   11.1
     Back pain
    6.3
   8.8
     Arthralgia
    5.8
   9.0
     Diarrhea
    4.9
   5.5
     Hypertension
    4.7
   5.1
     Constipation
    4.5
   6.0
     Pain in extremity
    4.1
   3.2
     Anemia
    4.0
   4.4
     Hot flush
    3.7
   4.1
     Nausea
    3.6
   5.6
     Weight loss
    2.5
   2.1
     Falls
    2.7
   4.1
     Bone fracture
    3.0
   3.5
     Memory impairment
    0.4
   1.2
     Cognitive disorder
    0.3
   0.2
 Seizure
    0.2
    0.2
                  Metastatic Castration-Resistant Prostate Cancer (mCRPC)
Darolutamide Safety Data
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 ARADES Trial 5 Study Design
The ARADES trial, an open-label, multicenter trial with a non-randomized Phase I dose escalation portion and a Phase II randomized dose expansion phase, enrolled patients with progressive, metastatic castration-resistant prostate cancer (mCRPC). Patients with a history of seizures or were at risk of seizures were included.5 
Patients (N = 134) in the ARADES trial had progressive, mCRPC with serum testosterone levels <30 ng/m and were either chemotherapy-naïve or had ≤2 prior chemotherapy regimens.5 
Baseline characteristics are presented in Table 6.5  Table 6: Baseline Characteristics.5 
        Total N=134
   CYP17i-naïve, chemo-naïve, N
     42 (31.3)
   CYP17i-naïve, post-chemo
     35 (26.1)
   Post-CYP17i
     57 (42.5)
   Age, median years (range)
     60 (53-89)
   PSA ng/mL, median (range)
     99 (3-5000)
   Time from diagnosis to study treatment, median months (range)
    69 (10-231)
 Disease localization, N(%): Bone disease only
Soft Tissue only
Soft and Bone Disease
   49 (36.6) 20 (14.9) 64 (47.8)
   Seizure (and like) event history %
    11 (8.2)
 Time on Study Drug (days) Mean (SD)
Median (min, max)
  184 (131.0) 168 (1,684)
    Safety Results 5
The most common adverse events (all grades) were fatigue including asthenia (31%, arthralgia (16%) and pain (15%). Most adverse events reported were Grade 1 or 2. Grade 3 and 4 adverse events reported in 22% (n = 27) and <2% (n = 2) patients, respectively. Darolutamide was tolerated up to the highest dose studied (1800 mg/day) with the maximum tolerated dose was not reached. No seizures were reported.3    See table 7 below.
Adverse events considered drug-related were reported in 35% (n = 44) patients and included fatigue including asthenia in 12% (n = 15) of patients, hot flushes in 5% (n = 6) of patients, decreased appetite in 4% (n = 5) of patients and diarrhea in 2% (n = 3) of patients. One grade 3 event (fatigue including asthenia) was deemed by the investigator to be possibly drug-related. No grade 4 events were deemed to be drug-related.5 
Darolutamide Safety Data
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 Table 7
    ARADES: TEAEs with an Incidence of  ≥ 10% (n = 134) – darolutamide 200 – 1800 mg/day5 
  Adverse Event
    Grade 1
     Grade 2
     Grade 3
    Total*
     Fatigue including Asthenia
 30 (22%)
 13 (10%)
 2 (1%)
 41 (31%)
     Back pain
    17 (13%)
    13 (10%)
    2 (1%)
   28 (21%)
     Arthralgia
    18 (13%)
    9 (7%)
    2 (1%)
   22 (16%)
     Pain
    9 (7%)
    8 (6%)
    3 (2%)
   20 (15%)
     Constipation
    15 (11%)
    5 (4%)
    0
   18(13%)
     Nausea
    14 (10%)
    4 (3%)
    0
   17 (13%)
     Decreased appetite
    15 (11%)
    1 (<1%)
    0
   16 (12%)
 Peripheral edema
    14 (10%)
     2 (1%)
     0
    16 (12%)
      *Total may be less than the sum of the grade because patients may have had adverse events of several grades.
Darolutamide Safety Data
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 ARAFOR Trial 6 Study Design
ARAFOR, a two-part, multicenter, international, open-label, Phase I study, evaluated the pharmacokinetic (Pk), safety and tolerability of darolutamide (ODM-201) in patients (N = 30) with progressive metastatic castration resistant prostate cancer (mCRPC). Patients eligible for the study had to meet the following inclusion criteria: ≥18 years of age with progressive mCRPC, testosterone level <1.7 nmol/L, ECOG performance status of 0-1, chemotherapy naïve, and asymptomatic or mildly symptomatic. Patients without bilateral orchiectomy were to continue on gonadotropin-releasing hormone therapy. Patients with brain metastases or previous treatment with a second-generation Androgen Receptor antagonist or CYP17 inhibitor was excluded.6 
The first part of the study (Pk analysis) was a randomized open-label, two arm, three-period crossover study. The second part of the study (safety analysis) was an open-label extension to evaluate long-term safety and tolerability. The primary endpoint of the first part of the study was PK of tablet products compared to the capsule formulation and effect of food. The endpoints in the second part of the study were safety and effects on PSA and lesions.6 
Safety Results 6
After a median reporting period of 15.3 months (95% CI 9.7 months – NR), 73% (22/30) of patients reported adverse events. The majority of the AE’s reported were Grade 1 or 2. The most common AEs were reported included fatigue (13%, n = 4) [All grade 1] and nausea (13%, n = 4) [All grade 1 – 3]. Twenty percent (20%) of AEs reported were considered treatment-related. Treatment-related AEs were fatigue (13%), decreased appetite, headache, abdominal pain, solar dermatitis, tinnitus, and dysgeusia. All treatment-related AEs were Grade 1. No seizures were reported6 
No dose reductions were necessary. One patient (3.3%) died due to cancer progression. Two patients (6.7%) discontinued due to an AE (neuroendocrine carcinoma and respiratory failure). These AEs were not consider treatment-related by the investigator.6    For AEs reported in >5% of patients, see table 8 below.
Darolutamide Safety Data
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

Table 8. Adverse Events occurring in >5% of patients by Grade6  (Safety Population, N = 30)
Adverse event
Grade 1 n (%)
Grade 2 n (%)
Grade 3 n (%)
Total n (%)
Nausea
3 (10)
1 (3)
1 (3)
4 (13)
Fatigue
4 (13)
0
0
4 (13)
Bone pain
1 (3)
2 (7)
1 (3)
3 (10)
Abdominal pain
3 (10)
0
0
3 (10)
Back pain
0
2 (7)
1 (3)
3 (10)
Diarrhea
2 (7)
1 (3)
0
3 (10)
Hematuria
2 (7)
1 (3)
0
3 (10)
Erysipelas
2 (7)
1 (3)
0
2 (7)
Pain in extremity
2 (7)
1 (3)
0
2 (7)
Cough
2 (7)
0
0
2 (7)
Dysuria
2 (7)
0
0
2 (7)
Headache
2 (7)
0
0
2 (7)
Hypertension
0
2 (7)
0
2 (7)
Influenza
2 (7)
0
0
2 (7)
Rhinorrhea
2 (7)
0
0
2 (7)
Rib fracture
2 (7)
0
0
2 (7)
Skin ulcer
2 (7)
0
0
2 (7)
Solar dermatitis
2 (7)
0
0
2 (7)
REV 0719 References/ Enclosure(s): (Bolded)
1. NUBEQA®    (darolutamide) tablets for oral use [Prescribing Information]. Whippany, NJ. Bayer HealthCare Pharmaceuticals, July 2019.
2. Institute for Clinical and Economic Review (ICER), 2018
3. Fizazi K, et al. Darolutamide in nonmetastatic, castration-resistant prostate cancer). Oral presentation at: [ASCO
GU]; February 14 – 16, 2019; San Francisco CA. Presentation number #GU19.
4. Fizazi K, Shore N, Tammela TL, et. Darolutamide in nonmetastatic castration-resistant prostate cancer;  N Eng J
M, 20  19, PP 1 – 12,DOI: 10.1056/NEJMMoa1815671
5. Fizazi K, et al. Activity and safety of odm-201 in patients with progressive metastatic castration-resistant
prostate cancer (ARADES): an open-label phase I dose-escalation and randomized phase II dose expansion
trial.  Lancet Oncol , 2014;15:975-985
6. Massard C, et al. Pharmacokinetics, antitumor activity, and safety of ODM-201 in patients with
chemotherapy-naïve metastatic castration-resistant prostate cancer: an open-label phase I study.  Eur Urol
2016;69:834-840.
7. Pajouhesh H, Lenz GR.  NeuroRx . 2005;2:541–553
Darolutamide Safety Data
Disclaimer -  This information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.""")

In [114]:
displacy.render(doc, style="ent", options=options)

In [115]:
displacy.serve(doc, style="ent", options=options)

/Users/subashgandyer/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [05/Mar/2020 10:59:26] "GET / HTTP/1.1" 200 87340
127.0.0.1 - - [05/Mar/2020 10:59:27] "GET /favicon.ico HTTP/1.1" 200 87340


Shutting down server on port 5000.


### Document 5 - Nubeqa (US-NUBEQA-76050-S.docx-2.pdf)

In [116]:
doc = loaded_nlp("""NUBEQA®   (darolutamide) Pharmacokinetics
● Nubeqa (darolutamide) is an androgen receptor inhibitor indicated for the treatment of patients with non-metastatic castration-resistant prostate cancer.1 
● Pharmacokinetics of darolutamide were determine as a part of the phase1-2 study, which determined dose escalation, dose expansion, and long term follow-up.1 
o Pharmacokinetics were determined in patients receiving 200 mg/day (n=3), 400 mg/day (n=7), 600 mg/day (n=3) 1000 mg/day (n=3), 1400 mg/day (n=3), 1800 mg/day (n=3) with food. No intra-patient escalation was permitted.
o Pharmacokinetic analysis of darolutamide showed rapid absorbed and the median t max  ranged 3.0–5.1 hours for darolutamide and 1.5–5.0 h for diastereo isomer keto-darolutamide on day 1. The mean metabolite to parent ratio (area under the curve, AUC) at steady state was 1.6–2.3:1. The exposure of darolutamide (AUC t a  nd C max)  at steady state increased in a linear, dose-related manner up to 1400 mg daily dose.
o Most common Grade 3 adverse events occurring in  >  10% of patients were fatigue or asthenia 2 (2%), back pain 2 (2%) and arthralgia 2 (2%).
● An open randomized phase 1 study evaluated safety and tolerability, bioavailability and effect of food on bioavailability of darolutamide in patients with metastatic chemotherapy naïve CRPC.2    The study compared relative bioavailability of 600 mg tablets (tablet A and tablet B) and capsule (reference) formulations in patients with chemotherapy naïve metastatic castrate resistant prostate cancer.
o Patients were randomized to one of 3 treatment sequences in each group.   There were three treatment periods for each patient during the PK Component which included administration of two darolutamide 300 mg test tablet (Tab A or Tab B) in both fasted and fed conditions, or administration of darolutamide 100 mg capsule at 600 mg dose in fed state. There was at least 7 days wash-out period between the three study treatment administrations. The total daily dose was 600 mg/day
o Thirty men with were enrolled. Area under the concentration curve (AUC) and maximum concentration (C max)  values were approximately two times lower after administration at fast. With high fat meal the C max  occurred about 2-3 hours later. The food effect was similar in all groups.
o The pharmacokinetics observed with the two new tablet formulations were comparable with the pharmacokinetics of the capsule formulation
Nubeqa (darolutamide) is an androgen receptor inhibitor indicated for the treatment of patients with non-metastatic castration-resistant prostate cancer.Phase 1-2 study
Phase 1–2 clinical trial was conducted to assess darolutamide in men with progressive metastatic castration-resistant prostate cancer.1    The primary objectives of the trial were to
Darolutamide Pharmacokinetics
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
 
 assess safety and tolerability and to define a maximum tolerated dose. Secondary objectives were to determine the pharmacokinetics, and antitumor activity as defined by changes in serum prostate-specific antigen (PSA), by imaging of soft tissue and bone lesions, and by changes in circulating tumor cell counts. Tolerability and outcome measures are discussed elsewhere.
Blood samples were collected for the pharmacokinetics analyses. We identified plasma concentrations of diastereoisomers [S,R]-darolutamide and [S,S]-darolutamide, and the major metabolite keto-darolutamide by liquid chromatography-tandem mass spectrometry (LC-MS/MS); Darolutamide concentration was the sum of [S,R]-darolutamide and [S,S]-darolutamide concentrations.1 
Pharmacokinetic analysis of darolutamide showed rapid absorbed and the median t max  were 3·0–5·1 h for darolutamide and 1.5–5.0 h for keto-darolutamide on day 1.1    The mean metabolite to parent ratio (area under the curve, AUC) at steady state was 1.6–2.3:1. The exposure of darolutamide (AUC t  and C max)  at steady state increased in a linear, dose-related manner up to 1400 mg daily dose. Dose escalation was discontinued because of darolutamide plasma concentration plateau. Steady-state plasma concentrations were reached after 1 week of continuous treatment. The mean half-life of darolutamide independent of dose was 15.8 hours (SD 13.8) at steady state after 200–1800 mg daily doses. The mean half-life of keto-darolutamide was 10.0 hours (3.8) at steady state after 200–1800 mg daily doses.
Mean  +  standard deviations steady state concentrations at various doses are presented in Figure 1.1 
Figure 1: Mean  +  SD Concentrations of DAROLUTAMIDE 1
Darolutamide mean C max  by dose are presented in Figure 2a, and mean  +  SD areas under concentration curve (AUC) of darolutamide is presented in Figure 2b.1 
Darolutamide Pharmacokinetics
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
    
  Figure 2a: Mean  +  SD C max  Based on Dose1 
   Figure 2b: AUC of Darolutamide by Dose
          Most common Grade 3 adverse events occurring in  >  10% of patients were fatigue or asthenia 2 (2%), back pain 2 (2%) and arthralgia 2 (2%).
Phase I bioavailability study
An open randomized phase 1 trial (ARAFOR) evaluated bioavailability and effect of food on bioavailability of darolutamide.2    The study compared relative bioavailability of 600 mg tablets (tablet A and tablet B) and capsule (reference) formulations in patients with  chemotherapy naïve metastatic castrate resistant prostate cancer. The study also assessed  the effect of food on the bioavailability of darolutamide tablet products tablet A (Tab A) and tablet B (Tab B).
The PK Component consisted of two concurrent groups, which evaluated Tab A and (Tab B), and capsule. Patients were randomized to one of 3 treatment sequences in each group.2     There were three treatment periods for each patient during the PK Component which included administration of two darolutamide 300 mg test tablets (Tab A or Tab B) in both fasted and fed conditions, or administration of darolutamide (6 x100 mg) capsules at 600 mg dose in fed state. There was at least a 7 days wash-out period between the three study treatment administrations. The total daily dose was 600 mg/day
Thirty  men with metastatic chemotherapy-naïve CRPC were enrolled.2    Food interaction was seen when darolutamide formulations (600 mg) were administered after a high fat content breakfast compared to administration in fasted state. AUC and C max  values were approximately
Darolutamide Pharmacokinetics
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
 
two times lower after administration at fast. With high fat meal the C max  occurred about 2-3 hours later, indicating the delayed gastric emptying of darolutamide after the high fat meal. The food effect was similar in all groups. The pharmacokinetics observed with the two new tablet formulations were comparable with the pharmacokinetics of the capsule formulation (figure 4
Figure 4: Concentration Time Curves of the Three Formulation with and Without Food2 
Most commonly reported adverse events (>10%) were fatigue/asthenia (13%), nausea (10%), erythema (10%) and urinary tract disorder (10%); all being mild events except one nausea event (grade 3).
REV: 0719 Reference:
1. NUBEQA®    (darolutamide) tablets for oral use [Prescribing Information]. Whippany, NJ. Bayer HealthCare Pharmaceuticals, July 2019.
2. Fizazi K, Massard C, Bono P, et al. Activity and safety of ODM 201 in patients with progressive metastatic castration-resistant prostate cancer (ARADES): an open-label phase 1 dose-escalation and randomised phase 2 dose expansion trial.  Lancet Oncol  2014; 15: 975–85. http://www.ncbi.nlm.nih.gov/pubmed/24974051
3. Massard C, Tammela T, Vjaters, et al. A study of ODM 201 formulations with a safety and tolerability extension phase in patients with metastatic chemotherapy naïve castrate-resistant prostate cancer (CRPC). Presented at 2014 Genitourinary Cancers Symposium, January 2014 in San Francisco CA.
Darolutamide Pharmacokinetics
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.""")

In [117]:
displacy.render(doc, style="ent", options=options)

In [118]:
displacy.serve(doc, style="ent", options=options)

/Users/subashgandyer/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [05/Mar/2020 11:00:01] "GET / HTTP/1.1" 200 63514
127.0.0.1 - - [05/Mar/2020 11:00:01] "GET /favicon.ico HTTP/1.1" 200 63514


Shutting down server on port 5000.


### Document 6 - Nubeqa (US-NUBEQA-76051-S.docx-2.pdf)

In [119]:
doc = loaded_nlp("""ARAMIS: A Phase III Study of darolutamide in Men with High-risk Non-metastatic Castration-resistant Prostate Cancer
Summary
● Nubeqa®    (darolutamide) is an androgen receptor inhibitor indicated for the treatment of patients with non-metastatic castration-resistant prostate cancer.1 
● ARAMIS, a Phase III, randomized, double-blind, placebo-controlled multinational study, evaluated the efficacy and safety of darolutamide in men being treated with androgen deprivation therapy (ADT*) for non-metastatic castration-resistant prostate cancer (nmCRPC) with a high risk of developing metastases.1    Patients (N = 1509) were randomized 2:1 to receive darolutamide 600 mg tablets orally twice daily plus ADT* versus matching placebo plus ADT*. The primary endpoint of the study was metastasis-free survival (MFS). MFS was defined as the time from randomization to evidence of metastasis (new or progressive regional pathological lymph nodes were not defined as metastasis) or death from any cause, whichever occurred first. Secondary endpoints included overall survival (OS), Time to Pain Progression (TTPP), Time to Initiation of Cytotoxic Chemotherapy and Time to First Symptomatic Skeletal Event (SSE) and Safety and tolerability. Exploratory endpoints included Progression-free Survival (PFS), and Time to PSA Progression.2 
Primary Endpoint
o Darolutamide demonstrated a statistically significant improvement in MFS of 40.4 months [95% CI 34.33 – NR] vs 18.4 months [95% CI 15.51 – 22.34] in the placebo arm and reduced the risk of metastasis or death by 59% versus placebo [HR .41 (95% CI 0.34 – 0.50);  P  <0.001].4 
Secondary Endpoints
o At interim analysis, the secondary endpoint of median overall survival (OS) was not reached [NR;  P-  value = 0.045; HR 0.71 (95% CI 0.50 – 0.99)]. The  P -value did not reach the pre-defined threshold of 0.0005 for statistical significance.
▪ The significance of all other secondary endpoints was dependent on OS meeting statistical significance as per hierarchal methodology.
o Time to Pain Progression (TTPP) was 40.31 months [95% CI 33.2 – 41.2] in the darolutamide arm vs. 25.36 months [95% CI 19.09 – 29.36] in the placebo arm. The HR of 0.65 [(95% CI 0.53 – 0.79);  P  <0.001] demonstrated a 35% risk reduction in the time to pain progression compared to placebo.4 
o The median Time to Initiation of First Cytotoxic Chemotherapy was NR in the darolutamide arm vs 38.2 months (95% CI 35.55 – 41.89) in the placebo arm. The HR of 0.43 [(95% CI 0.31 – 0.60);  P  <0.001] demonstrated a 57% risk reduction in the time to initiation of First Cytotoxic Chemotherapy compared to placebo.4 
The median Time to First Symptomatic Skeletal Event was not reached in either arm. At the time of analysis, 1.7% (16/955) of patients in the darolutamide arm experienced a SSE versus 3.2% (18/554) of patients in the placebo arm. The HR 0.43 [(95% CI 0.22 – 0.84),  P=  0.011] demonstrated a 57% risk reduction in the Time to First Symptomatic Skeletal Event compared to placebo.4 
NUBEQA ®  (darolutamide): ARAMIS (nmCRPC)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
o
 * Common previous hormonal therapies for prostate cancer (received by ≥10% of all patients) included leuprolide (52%), goserelin (32%), triptorelin (29%), bicalutamide (66%), flutamide (13%), and cyproterone (11%).
Exploratory Endpoints
o The median PFS was 36.8 months (95% CI 32.92 – NR) in the darolutamide arm versus 14.8 months (95% CI 11.83 – 14.33) in the placebo arm;  P <0.001.4 
o The median Time to PSA Progression was 33.2 months (95% CI 25.89 – NR) in the darolutamide arm versus 7.3 months (95% CI 13.94 – 7.39) in the placebo arm;
P<  0.001; HR 0.13 (95% CI 0.11 – 0.16) demonstrating an 87% decrease in the risk of PSA Progression compared to placebo.4 
Safety
o After a median exposure of 14.8 months in the darolutamide arm versus 11.04 months in the placebo arm, approximately 80% of patients experienced an adverse event during the trial (N = 1220/1508; [darolutamide arm, n = 794; placebo arm, n = 426]). The majority of adverse events reported in both arms were Grade 1 or 2 [54.4% (821/1508)].2    The incidence of common adverse events were reported in ≤10% of patients in both the darolutamide and placebo arms, with only fatigue (including asthenia) reported at an incidence of >10% in both arms.3 
o The most common Grade 3 or 4 events were hypertension (3.1% of patients in the darolutamide plus ADT arm versus 2.2% of patients in the placebo plus ADT arm.) and urinary retention (1.6% of patients in the darolutamide plus ADT arm versus 2.0% of patients in the placebo plus ADT arm.).3 
o Treatment discontinuations occurred in 35.5% (339/954) of patients in the darolutamide arm vs. 63.9% (354/554) of patients in the placebo arm.2 
Clinical Data
Study Design 2
ARAMIS, a Phase III, multinational, randomized, double blind, placebo-controlled study, evaluated the efficacy and safety of darolutamide in men undergoing treatment with androgen deprivation therapy (ADT*   ) for non-metastatic castration-resistant prostate cancer (nmCRPC) with a high risk of developing metastases. High risk was defined as PSADT of ≤ 10 months and PSA ≥ 2ng/ml. Patients included in the study had an ECOG performance status of 0-1 and adequate hematologic, renal, and liver function. Additionally, patients with a history of seizures or at risk for seizures were included.2 
Patients (N=1509) were randomized 2:1 to receive darolutamide 600 mg tablets orally twice daily with food plus ADT versus matching placebo plus ADT. Key baseline characteristics of the majority of the patient population in the ARAMIS study were: a median age of 74; Gleason score of ≥7; absence of regional pathological lymph nodes; ≥2 prior hormonal therapy agents with a PSADT ≤6 months, and no baseline osteoclast targeted therapy. Patients on the ARAMIS study were stratified according to PSADT (PSADT;  <  6 months vs. > 6 months) and use of
NUBEQA ®  (darolutamide): ARAMIS (nmCRPC)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
 
 osteoclast-targeted therapy (yes vs. no). All assessments were obtained at baseline and every 16 weeks. For a complete detail of patient demographics and characteristics, see table 1 and 2 below.2 
    Table 1. ARAMIS: Patient Demographics3 
        Darolutamide N = 955
   Placebo N = 554
  Total N = 1509
 Age , median, years
    74.0
     74.0
    74.0
     Age group, n (%)
     <65
   113 (11.8)
   84 (15.2)
  197 (13.1)
     65 - 74
    373 (39.1)
    216 (39.0)
   589 (39.0)
     75 – 84
    384 (40.2)
    209 (37.7)
   593 (39.3)
 ≥85
    85 (8.9)
     45 (8.1)
    130 (8.6)
     Race, n (%)*
  Asian
    122 (12.8)
     71 (12.8)
    193 (12.8)
     Black or African-American
 28 (2.9)
 24 (4.3)
 52 (3.4)
     Other
    9 (0.9)
    6 (1.1)
   15 (1.0)
 White
    760 (79.6)
     434 (78.3)
    1194 (79.1)
     Weight
     N
   953
   552
  1505
     Missing, n
    2
    2
   4
 Median, kg
    82.40
     82.00
    82.00
     Geographical region, n (%)
     North America
   108 (11.3)
   76 (13.7)
  184 (12.2)
     Asia Pacific
    119 (12.5)
    67 (12.1)
   186 (12.3)
 Rest of the world
    728 (76.2)
     411 (74.2)
    1139 (75.5)
                 *Race data was missing for 36 (3.8%) and 19 (3.4%) of patients in the darolutamide and placebo arms, respectively
    Table 2. ARAMIS: Patient Characteristics3 
        Darolutamide N = 955
   Placebo N = 554
  Total N = 1509
     Base line PSADT ≤6 months, n (%)
    667 (69.8)
    371 (67.0)
   1038 (68.8)
     PSADT, months, mean (median)
    4.84 (4.39)
    4.89 (4.65)
   4.86 (4.45)
 PSA, ng/mL, mean (median)
     18.7 (9.0)
      19.8 (9.7)
     19.1 (9.3)
   No baseline osteoclast-target therapy, n (%)
  924 (96.8)
   522 (94.2)
  1446 (95.8)
     ECOG PS, n (%)
     0
   650 (68.1)
   391 (70.6)
  1041 (69.0)
 1
    305 (31.9)
     163 (29.4)
    468 (31.0)
     Gleason total score (Factor1 + Factor2), n (%)
     Missing
   27 (2.8)
   17 (3.1)
  44 (2.9)
 <7
     217 (22.7)
      142 (25.6)
     359 (23.8)
   ≥7
  711 (74.5)
   395 (71.3)
  1106 (73.3)
     Prior hormonal therapy, n (%)103 (18.6)
           NUBEQA ®  (darolutamide): ARAMIS (nmCRPC)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

      1
   177 (18.5)
   103 (18.6)
  280 (18.6)
     ≥2
    727 (76.1)
    420 (75.8)
   1147 (76.0)
 Not applicable*
    51 (5.3)
     31 (5.6)
    82 (5.4)
     Baseline presence of regional pathological lymph nodes by central imaging review, n (%)
     No
   792 (82.9)
   396 (71.5)
  1188 (78.7)
 Yes
    163 (17.1)
     158 (28.5)
    321 (21.3)
     *Patients who underwent surgical castration were not required to have been/ be treated with hormonal drug therapy
Efficacy Results3  Primary Endpoint: Metastasis Free Survival
After a median follow-up of 17.9 months, darolutamide demonstrated a statistically significant improvement in MFS of 40.4 months [95% CI 34.33 – NR] vs 18.4 months [95% CI 15.51 – 22.34] in the placebo arm and reduced the risk of metastasis or death by 59% compared to placebo [HR .41 (95% CI 0.34 – 0.50);  P  <0.001]. This improvement in MFS was consistent among patients irrespective of baseline characteristics, therapy and demographic See table 3 and figures 1 & 2 below.3 
     Table 3. ARAMIS: Metastasis-Free Survival (MFS)3 
        Darolutamide
  Placebo
     N
    955
   554
 Events, n (%)
     221 (23.1)
     216 (39.0)
     Median, months (95% CI)
 40.37 (34.33 – NR)
 18.43 (15.51 – 22.34)
     Baseline metastases, n (%)
    50 (5.2)
    39 (7.0)
      2-sided  P -value
  <0.001
 HR (95% CI)
   0.41 (0.34 – 0.50)
        NUBEQA ®  (darolutamide): ARAMIS (nmCRPC)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 Figure 1 Kaplan-Meier Analysis of Metastasis-free Survival3 
 NUBEQA ®  (darolutamide): ARAMIS (nmCRPC)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 Figure 2    Subgroup Analysis of Metastasis-free Survival3 
 Secondary Endpoints4  Overall Survival
Due to the data being immature at the time of analysis, the P-value for OS did not reach the pre-defined threshold for statistical significance [OS NR in either arm, P-value 0.045 (HR 0.71; 95% CI, 0.50 to 0.99)]. As per hierarchal methodology, unless statistical significance was reached for OS, all other secondary endpoints would not be considered to be significant. Figure 3 below show the results of the overall survival analysis (a secondary endpoint).4 
 NUBEQA®    (darolutamide): ARAMIS (nmCRPC)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 Figure 3 ARAMIS: Overall Survival (OS) KM Curve3 
 Time to Pain Progression
Darolutamide demonstrated an improvement in Time to Pain Progression (TTPP) of 40.3 months [95% CI 33.2 – 41.2] vs.25.4 months [95% CI 19.09 – 29.36] in the placebo arm; HR .65 (95% CI 0.53 – 0.79);  P  <0.001. TTPP was identified by an increase of ≥2 points from baseline in question three of the BPI-SF [related to the worst pain in the previous 24 hours] take as a 7-day average or initiation of short or long-acting opioids for pain, whichever comes first).4    See figure 4 below.
 NUBEQA®    (darolutamide): ARAMIS (nmCRPC)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 Figure 4 ARAMIS: Time to Pain Progression (TTPP) KM Curve3 
 Time to Initiation of First Cytotoxic Chemotherapy
Darolutamide demonstrated a significant difference in the Time to Initiation of First Cytotoxic Chemotherapy when compared to placebo. At the time of analysis, 35% (339/955) of patients in the darolutamide arm had terminated treatment versus 64% (354/554) of patients in the placebo arm. Of the patients who terminated treatment in the darolutamide arm, 29.5% (100/339) vs 36.7% (130/354) of patients in the placebo arm, received cytotoxic chemotherapy and/or anti-neoplastic therapy. The median time to initiation of first cytotoxic chemotherapy was 38.2 months (95% CI 35.55 – 41.89) in the placebo arm and not yet reached in the darolutamide arm.; HR 0.43 (0.31 – 0.60), P<0.001.4    See table 4 below.
Time to First Symptomatic Skeletal Event (SSE)
The time to first symptomatic skeletal event was evaluated between the two groups. At the time of analysis, 1.7% (16/955) of patients in the darolutamide arm experienced a SSE versus 3.2% (18/554) of patients in the placebo arm. The median time to first symptomatic skeletal event was not reached in either arm; HR 0.43 (95%CI 0.22 – 0.84),  P=  0.01.3    See table 4 below.
NUBEQA®    (darolutamide): ARAMIS (nmCRPC)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
  
 Exploratory Endpoints4  Progression-Free Survival (PFS)
PFS was defined as the time between randomization and evidence of any radiographic disease progression, including new pathologic lymph nodes identified above or below the aortic bifurcation, or death from any cause, whichever occurs first.3 
At the time of analysis, 26.8% (256/955) events had occurred in the darolutamide arm versus 46.8% (259/554) of events in the placebo arm. The median PFS was 36.8 months (95% CI 32.92 – NR) in the darolutamide arm versus 14.8 months (95% CI 11.83 – 14.33) in the placebo arm; HR 0.38 (0.32 – 0.45),  P<  0.001.4    See table 4 and figure 5 below.
Figure 5 ARAMIS: Progression-Free Survival KM Curve3 
Time to PSA Progression
Time to PSA progression was defined according to PCWG2 Consensus Guidelines.
At the time of analysis, 23.7% (226/955) events had occurred in the darolutamide arm versus 66.4% (368/554) of events in the placebo arm. The median Time to PSA Progression was 33.2 months (95% CI 25.89 – NR) in the darolutamide arm versus 7.3 months (95% CI 13.94 – 7.39) in the placebo arm;  P<  0.001; HR 0.13 (95% CI 0.11 – 0.16) demonstrating an 87% decrease in the risk of PSA Progression compared to placebo.4    See table 4 and figure 6 below.
NUBEQA®    (darolutamide): ARAMIS (nmCRPC)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
   
 Figure 6 ARAMIS: Time to PSA Progression3 
    Table 4 Pre-specified Secondary and Exploratory Efficacy End Points (Intention-To-treat)*4 
  End Point
 Darolutamide (N = 955)
 Placebo (N= 554)
  Hazard Ratio (95% CI)
 P Value
  Median Duration mo
    No. of Events
   Median Duration mo
   No. of Events
    Secondary end points Overall survival
Time to pain progression
Time to cytotoxic chemotherapy
Time to first symptomatic skeletal
event
    NR 40.3
NR NR
 78 251
73 16
NR 25.4
38.2 NR
    58 178
79 18
0.71 (0.50 – 0.99) 0.65 (0.53 – 0.79)
0.43 (0.31 – 0.60) 0.43 (0.22 – 0.84)
   0.045 <0.001
<0.001 0.01
 Exploratory end points Progression-free survival
Time to PSA progression
  36.8 33.2
    255 226
   14.8 7.3
   258 368
  0.38 (0.32 – 0.45) 0.13 (0.11 – 0.16)
   <0.001 <0.001
NUBEQA®    (darolutamide): ARAMIS (nmCRPC)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 NUBEQA®    (darolutamide): ARAMIS (nmCRPC)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

 Safety Results 3
After a median exposure of 14.8 months in the darolutamide arm versus 11.04 months in the placebo arm, approximately 80% of patients experienced an adverse event during the trial
(N = 1220/1508; [darolutamide arm, n = 794; placebo arm, n = 426]). The incidence of adverse events were generally similar between treatment arms and the majority were Grade 1 or 2 [54.4% (821/5108)].3    The incidence of common events was low with only fatigue reported at an incidence of >10%. Grades 3 or 4 adverse events occurred in 22.8% (344/1508) of the total safety population (24.7% of patients in the darolutamide arm versus 19.5% of patients in the placebo arm). The most frequent Grade 3 or 4 adverse events included hypertension (3.1% of patients in the darolutamide plus ADT arm versus 2.2% of patients in the placebo plus ADT arm.) and urinary retention (1.6% of patients in the darolutamide plus ADT arm versus 2.0% of patients in the placebo plus ADT arm.).3 
Table 5 shows the Treatment Emergent Adverse Events (TEAEs) for the total safety population (N = 1508). TEAEs (all grades) occurring in ≥5% of patients reported in Table 6 below, in the darolutamide versus placebo arms, respectively, included fatigue (including asthenia) (12.1% [115/954, vs 8.7% [48/554]). Other TEAEs occurring in ≥5% in the darolutamide arm included back pain (8.8%), arthralgia (8.1%), diarrhea (6.9%), hypertension (7.3%), constipation (TEAEs occurring in 6.3%), pain in extremity (5.8%), anemia (5.6%) and hot flush (5.2%). Approximately 14% of TEAEs led to a dose modification in the darolutamide arm versus 9.4% of TEAEs in the placebo arm.3 
After a median follow-up of 17.9 months, treatment discontinuations were similar between both arms.4    Discontinuations due to adverse events occurred in 8.9% of patients in the darolutamide arm vs 8.7% of patients in the placebo arm. Discontinuations due to metastasis occurred in 11.7% vs 23.3% of patients; and discontinuations due to investigator judgment occurred in 5.7% vs 16.4% of patients, in the daroluamide vs placebo arms, respectively. Discontinuations due to protocol deviation and to other reasons unspecified occurred in 1.4% vs 1.3% and 0.6% vs 0.4% in the darolutamide vs placebo arm, respectively. Additionally, 7.1% vs 14.1% of patients in the darolutamide vs placebo arm respectively decided not to continue trial participation. It should be noted that one person in the darolutamide arm was randomized but did not receive treatment.3 
A total of 55 deaths occurred during the trial with 3.9% of deaths occurring in the darolutamide arm versus 3.2% occurring in the placebo arm.3    See Table 5 below.
The Institute for Clinical and Economic Review (ICER) has identified events such as falls, fractures, cardiovascular events and deaths due to AE’s, etc. to be TEAE’s of interest for anti-androgen therapies.6    For the incidence of TEAE’s of interest reported for darolutamide, see Tables 7 - 8.6 
NUBEQA®    (darolutamide): ARAMIS (nmCRPC)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.
    Table 5. ARAMIS: Summary of TEAEs3 
      Darolutamide N = 954
     Placebo N = 554
  Total N = 1508
     Exposure, median (months)
 14.8
      11.04
    Any AE, n (%)
  794 (83.2)
   426 (76.9)
   1220 (80.9)
    
      Worst grade
             Grade 1 or 2
 521 (54.6)
      300 (54.2)
   821 (54.4)
 Grade 3 or 4
   236 (24.7)
    108 (19.5)
    344 (22.8)
     Grade 5
37 (3.9)
    18 (3.2)
 55 (3.6)
     Serious
 237 (24.8)
      111 (20.0)
   348 (23.1)
 Leading to dose modification
   135 (14.2)
    52 (9.4)
    187 (12.4)
   Leading to permanent discontinuation
85 (8.9)
  48 (8.7)
 133 (8.8)
       AE, adverse event, TEAE, treatment adverse event
    Table 6. ARAMIS: Total of TEAEs with an Incidence of  ≥ 5%3 
     Preferred term, n (%)
   Darolutamide N = 954
  Placebo N = 554
     Fatigue
    115 (12.1)
   48 (8.7)
     Back pain
    84 (8.8)
   50 (9.0)
     Arthralgia
    77 (8.1)
   51 (9.2)
     Diarrhea
    66 (6.9)
   31 (5.6)
     Hypertension
    70 (7.3)
   33 (6.0)
     Constipation
    60 (6.3)
   34 (6.1)
     Pain in extremity
    55 (5.8)
   18 (3.2)
     Anemia
    53 (5.6)
   25 (4.2)
     Hot flush
    50 (5.2)
   23 (4.2)
     Nausea
    48 (5.0)
   32 (5.8)
     Bone Fracture
    40 (4.2)
   21 (3.8)
     Fall
    40 (4.2)
   26 (4.7)
     Rash
    24 (2.6)
   4 (0.7)
 Weight decreased
    34 (3.6)
    12 (2.2)
                    Table 7. ARAMIS: TEAEs of Interest3 
    Darolutamide N=954
Placebo N=554
                        AE
Any Grade, n (%)
Grade 3, %
Grade 4 %
Grade 5, %
Any Grade, n (%)
Grade 3, %
Grade 4 %
Grade 5, %
                                Bone Fracture
40 (4.2)
0.9
0
0
21 (3.8)
0.9
0
0
                    Fall
40 (4.2)
0.8
0
0
26 (4.7)
0.7
0
0
                      Fatigue, including asthenia
151 (15.8)
0.6
0
0
63 (11.4)
1.1
0
0
                    Seizure
2 (0.2)
0
1 (0.2)
0
                      Cognitive disorder
4 (0.4)
0
0
0
1 (0.2)
0
0
0
                                Memory impairment
5 (0.5)
0
0
0
7 (1.3)
0
0
0
                                Disturbance in attention
0
0
0
0
1 (0.2)
0
0
0
                                Amnesia
1 (0.1)
0
0
0
3 (0.5)
0
0
0
                    Rash
24 (2.6)
0.1
0
0
4 (0.7)
0
0
0
                      Weight decreased
34 (3.6)
0
0
0
12 (2.2)
0
0
0
                    Hot flush
50 (5.2)
0
-
-
23 (4.6)
0
-
-
                 Increased bilirubin
18 (1.9)
0.1
0
0
0
0
0
0
                 NUBEQA®    (darolutamide): ARAMIS (nmCRPC)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

     Table 8. ARAMIS: TEAEs of Interest3    continued
    Darolutamide N=954
Placebo N=554
                        AE
Any Grade, n (%)
Grade 3, %
Grade 4 %
Grade 5, %
Any Grade, n (%)
Grade 3, %
Grade 4 %
Grade 5, %
                    Hypertension
70 (7.3)
3.4
0
0
33 (6.0)
2.3
0
0
                      Cerebral and intracranial hemorrhage
2 (0.2)
0
0
0
2 (0.4)
0.2
0
0.2
                    Cerebral infarction and stroke
4 (0.4)
0.3
0
0
4 (0.7)
0.4
0
0.2
                      Cerebral ischemic infarction and stroke
5 (0.5)
0.1
0.2
0.1
2 (0.4)
0.2
0
0.2
                                Acute myocardial infarction
5 (0.5)
0.2
0.2
0.1
1 (0.2)
0
0
0
                    Cardiac failure
18 (1.9)
0.1
0.1
0.3
5 (0.9)
0
0
0.5
                 Coronary artery disorders
31 (3.2)
1.3
0.4
0.3
14 (2.5)
0.4
0
0.2
               Ischemic coronary artery disorders
24 (2.5
0.7
0.4
0.2
11 (2.0)
0.4
0
0.2
             QOL Assessment3 
Patient Reported Outcome tools (EORTC Q-PR25, Fact-P PCS and The Brief Pain Inventory – Short Form) were used to assess darolutamide’s impact on health-related quality of life versus placebo. Patient-Reported QoL was similar for darolutamide versus placebo, although clinically meaningful thresholds were not reached.5 
REV 0719
Reference
1. NUBEQA® (darolutamide) tablets for oral use [Prescribing Information]. Whippany, NJ. Bayer HealthCare Pharmaceuticals, July 2019.
2. Data on file. Bayer HealthCare Pharmaceuticals, Whippany, NJ.
3. Fizazi K et al. ARAMIS: efficacy and safety of darolutamide in nonmetastatc castration-resistant
prostate cancer Oral presentation at: [ASCO GU]; February 14 – 16, 2019; San Francisco CA.
#GU19.
4. Fizazi K, Shore N, Tammela TL, et. Darolutamide in nonmetastatic castration-resistant prostate
cancer;  N Eng J M, 2 019, PP 1 – 12,DOI: 10.1056/NEJMMoa1815671
5. Fizazi K, Shore N, Tammela TL, et al. efficacy and safety of darolutamide in nonmetastatc
castration-resistant prostate cancer. Poster presented at: ASCO GU]; February 14 – 16, 2019; San Francisco CA. Poster 140.
NUBEQA®    (darolutamide): ARAMIS (nmCRPC)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.

6. Institute for Clinical and Economic Review (ICER), 2018
NUBEQA®    (darolutamide): ARAMIS (nmCRPC)
Disclaimer - T  his information is for individual educational purposes only and is not intended to provide medical advice. Please consult the full prescribing information before using or prescribing Bayer Healthcare products. Bayer HealthCare 100 Bayer Boulevard Bldg. 200. Whippany, NJ 07981.""")

In [120]:
displacy.render(doc, style="ent", options=options)

In [121]:
displacy.serve(doc, style="ent", options=options)

/Users/subashgandyer/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [05/Mar/2020 11:00:23] "GET / HTTP/1.1" 200 74218
127.0.0.1 - - [05/Mar/2020 11:00:23] "GET /favicon.ico HTTP/1.1" 200 74218


Shutting down server on port 5000.
